In [ ]:
import tifffile
import numpy as np
import matplotlib.pyplot as plt
import cv2 

# --- Load the TIFF ---
file_path = "C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.tif"  # Adjust path if needed
img = tifffile.imread(file_path)

print("Array shape:", img.shape)      # Should be (69, 2, 748, 748)
print("Array dtype:", img.dtype)
print("Min/Max:", img.min(), img.max())

shape = img.shape
dtype = img.dtype

# --- Extract channels ---
img_ch1 = img[:, 0, :, :]   # Channel 1 (z, y, x)
img_ch2 = img[:, 1, :, :]   # Channel 2 (z, y, x)

# --- Plot z-slices ---
def plot_z_slices(img1, img2, num_slices=5):
    z_dim = img1.shape[0]
    step = max(1, z_dim // num_slices)
    fig, axs = plt.subplots(num_slices, 2, figsize=(7, 2 * num_slices))
    for i in range(num_slices):
        z = min(i * step, z_dim - 1)
        axs[i, 0].imshow(img1[z], cmap='gray')
        axs[i, 0].set_title(f'Ch1 z={z}')
        axs[i, 1].imshow(img2[z], cmap='gray')
        axs[i, 1].set_title(f'Ch2 z={z}')
        for j in range(2):
            axs[i, j].axis('off')
    plt.tight_layout()
    plt.show()

plot_z_slices(img_ch1, img_ch2, num_slices=5)

# Print shape and dtype
print("Array shape:", shape)
print("Array dtype:", dtype)
"""
#Middle Slices for Each Axis/Channel
z_mid = img.shape[0] // 2
ch_0 = img[z_mid, 0, :, :]
ch_1 = img[z_mid, 1, :, :]

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.imshow(ch_0, cmap='gray')
plt.title('Channel 0 (middle z)')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(ch_1, cmap='gray')
plt.title('Channel 1 (middle z)')
plt.axis('off')
plt.show()
"""
"""
#Explore Arbitrary Slices
z = 10  # change this to any value in [0,68]
plt.imshow(img[z, 0, :, :], cmap='gray')
plt.title(f'Ch0 z={z}')
plt.axis('off')
plt.show()

plt.imshow(img[z, 1, :, :], cmap='gray')
plt.title(f'Ch1 z={z}')
plt.axis('off')
plt.show()
"""
#Loop Over Z and Channel
# Visualize one z-slice from both channels, for several z values
for z in [0, img.shape[0]//4, img.shape[0]//2, 3*img.shape[0]//4, img.shape[0]-1]:
    fig, axs = plt.subplots(1,2, figsize=(8,4))
    axs[0].imshow(img[z, 0, :, :], cmap='gray')
    axs[0].set_title(f'Ch0 z={z}')
    axs[0].axis('off')
    axs[1].imshow(img[z, 1, :, :], cmap='gray')
    axs[1].set_title(f'Ch1 z={z}')
    axs[1].axis('off')
    plt.show()

In [ ]:
!pip install napari[all]

In [ ]:
%gui qt

import tifffile
import napari

# Load the image
img = tifffile.imread("C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.tif")  # Adjust path

# Shape: (z, channel, y, x) = (69, 2, 748, 748)
print('Shape:', img.shape)

# Transpose to (channel, z, y, x) for napari multichannel support (napari expects channels first!)
img_napari = img.transpose(1, 0, 2, 3)  # Now (2, 69, 748, 748)

# Launch napari
napari.view_image(
    img_napari,
    channel_axis=0,         # napari will treat axis 0 as channels
    name=['Channel 1', 'Channel 2'],
    contrast_limits=[(int(img.min()), int(img.max()))] * 2
)
napari.run()


In [ ]:
import numpy as np
from skimage import data

import napari
from napari.utils.translations import trans

import tifffile

viewer = napari.Viewer(ndisplay=3)

# add a 3D image
file_path = "C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"  # Adjust path if needed
data = tifffile.imread(file_path)

blobs = data.binary_blobs(
    length=64, volume_fraction=0.1, n_dim=3
).astype(np.float32)
image_layer = viewer.add_image(
    blobs, rendering='mip', name='volume', blending='additive', opacity=0.25
)

# add the same 3D image and render as plane
# plane should be in 'additive' blending mode or depth looks all wrong
plane_parameters = {
    'position': (32, 32, 32),
    'normal': (0, 1, 0),
    'thickness': 10,
}

plane_layer = viewer.add_image(
    blobs,
    rendering='average',
    name='plane',
    depiction='plane',
    blending='additive',
    opacity=0.5,
    plane=plane_parameters
)
viewer.axes.visible = True
viewer.camera.angles = (45, 45, 45)
viewer.camera.zoom = 5
viewer.text_overlay.text = trans._(
    """
shift + click and drag to move the plane
press 'x', 'y' or 'z' to orient the plane along that axis around the cursor
press 'o' to orient the plane normal along the camera view direction
press and hold 'o' then click and drag to make the plane normal follow the camera
"""
)
viewer.text_overlay.visible = True
if __name__ == '__main__':
    napari.run()

In [ ]:
!pip install czifile

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian
import napari
import pandas as pd

# ---- 1. LOAD .CZI IMAGE ----
file_path = "C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Try to find channel axis (adjust as needed!) ---
# Common: (Z, C, Y, X) or (C, Z, Y, X)
if img.shape[0] == 2:
    # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:
    # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

image = img_ch1  # or img_ch2, or sum

# ---- 2. SMOOTHING (reduces noise) ----
image_smooth = gaussian(image, sigma=1)

# ---- 3. BLOB DETECTION ----
blobs = blob_log(
    image_smooth,
    min_sigma=2,
    max_sigma=8,
    num_sigma=6,
    threshold=0.005,
    overlap=0.5
)
blobs[:, 3] = blobs[:, 3] * np.sqrt(3)   # sigma to radius

print(f"Detected {len(blobs)} lysosomes.")

# ---- 4. REGIONAL QUANTIFICATION (3D GRID) ----
# Set grid size (in voxels); e.g., divide Z, Y, X each into 4 bins
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

# Assign each blob to a region
z_idx = np.digitize(blobs[:, 0], z_bins) - 1
y_idx = np.digitize(blobs[:, 1], y_bins) - 1
x_idx = np.digitize(blobs[:, 2], x_bins) - 1

region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx

df = pd.DataFrame({
    "z": blobs[:, 0],
    "y": blobs[:, 1],
    "x": blobs[:, 2],
    "diameter_voxels": blobs[:, 3] * 2,
    "radius_voxels": blobs[:, 3],
    "region_id": region_labels
})

# Regional count/size summary:
region_summary = df.groupby("region_id").agg(
    count=("radius_voxels", "size"),
    avg_diameter=("diameter_voxels", "mean"),
    total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
).reset_index()

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# ---- 5. napari visualization ----
viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    name='Detected Lysosomes'
)
napari.run()

In [ ]:
!pip install imageio[ffmpeg]

In [ ]:
#FUNCIONA
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian
import napari
import pandas as pd
import imageio
import cv2

# ---- 1. LOAD .CZI IMAGE ----
file_path = "C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

image = img_ch1+img_ch2  # or img_ch2, or (img_ch1+img_ch2) for both

# ---- 2. SMOOTHING ----
image_smooth = gaussian(image, sigma=1)

# ---- 3. BLOB DETECTION ----
blobs = blob_log(
    image_smooth,
    min_sigma=1,      # Adjust if needed
    max_sigma=10,     # Adjust if needed
    num_sigma=8,
    threshold=0.005,   # Adjust for sensitivity
    overlap=0.5
)
blobs[:, 3] = blobs[:, 3] * np.sqrt(3)   # sigma to radius

print(f"Detected {len(blobs)} lysosomes.")

# ---- 4. REGIONAL QUANTIFICATION (3D GRID) ----
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

z_idx = np.digitize(blobs[:, 0], z_bins) - 1
y_idx = np.digitize(blobs[:, 1], y_bins) - 1
x_idx = np.digitize(blobs[:, 2], x_bins) - 1

region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx

# --- Calculate unique ID and volume for each blob ---
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3)

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0],
    "y": blobs[:, 1],
    "x": blobs[:, 2],
    "diameter_voxels": blobs[:, 3] * 2,
    "radius_voxels": blobs[:, 3],
    "volume_voxels": volumes,
    "region_id": region_labels
})

region_summary = df.groupby("region_id").agg(
    count=("radius_voxels", "size"),
    avg_diameter=("diameter_voxels", "mean"),
    total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
).reset_index()

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# ---- 5. NAPARI VISUALIZATION ----

viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')

# Color blobs by region for visual grouping
properties = {'region': region_labels.astype(int)}
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties=properties,
    face_color='region',
    face_colormap='tab20',
    name='Detected Lysosomes'
)


# ----Add region ID text labels at region centers ----
region_centers = []
region_ids = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)
region_ids = np.array(region_ids)

viewer.add_points(
    region_centers,
    size=8,
    face_color='black',
    name='Region Labels',
    text=[str(r) for r in region_ids]
)

img_for_video = image
if img_for_video.dtype != np.uint8:
    img_norm = ((img_for_video - img_for_video.min()) / (img_for_video.ptp()) * 255).astype('uint8')
else:
    img_norm = img_for_video

frames = [img_norm[z] for z in range(img_norm.shape[0])]

try:
    imageio.mimsave(
        'lysosome_stack.mp4', frames, fps=8, format='FFMPEG'
    )
    print('Saved: lysosome_stack.mp4')
except TypeError:
    print('MP4 export failed, trying GIF...')
    imageio.mimsave(
        'lysosome_stack.gif', frames, fps=8
    )
    print('Saved: lysosome_stack.gif')

frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame = img_norm[z].copy()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    # Draw all blobs that are at this z (or close to this z)
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
    for b in z_blobs:
        y, x = int(round(b[1])), int(round(b[2]))
        radius = int(round(b[3]))
        cv2.circle(frame_rgb, (x, y), radius, (0,255,255), 2)  # yellow
    frames_with_blobs.append(frame_rgb)

# Save with imageio (as mp4)
imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
print('Saved: lysosome_stack_blobs.mp4')

napari.run()

In [ ]:
#########################################Method GAUSSIAN-SATO#######################################333

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, sato
from skimage import filters, morphology, measure, exposure
from scipy import ndimage as ndi
import napari
import pandas as pd
import imageio
import cv2

# =========================
# 0) INPUT
# =========================
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

# =========================
# 1) LOAD .CZI IMAGE (robust axis handling)
# =========================
with czifile.CziFile(file_path) as czi:
    arr = czi.asarray()
    axes_raw = czi.axes  # e.g., 'TCZYX' (can include singleton axes)

# Remove singleton axes from both data and axes string
squeezed = np.squeeze(arr)
axes = ''.join(ax for ax, sz in zip(axes_raw, arr.shape) if sz > 1)
img = squeezed
print("Raw CZI axes:", axes_raw, "Raw shape:", arr.shape)
print("Squeezed axes:", axes, "Squeezed shape:", img.shape)

def find_axis(axchar):
    return axes.find(axchar) if axchar in axes else None

# indices of important axes
c = find_axis('C')
z = find_axis('Z'); y = find_axis('Y'); x = find_axis('X')
if None in (z, y, x):
    raise RuntimeError(f"Expected Z/Y/X in axes, got '{axes}'")

# Reorder to (Z, Y, X) or (Z, C, Y, X) if C exists
img_re = np.moveaxis(img, [z, y, x], [0, 1, 2])  # Z,Y,X are 0,1,2
if c is not None:
    # Adjust channel index after moving Z/Y/X out
    c_adj = c - sum(idx is not None and idx < c for idx in (z, y, x))
    # Bring channel to axis=1 => shape (Z, C, Y, X)
    img_re = np.moveaxis(img_re, c_adj, 1)
    if img_re.shape[1] < 2:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 0]  # duplicate if only one channel
    else:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 1]
else:
    img_ch1 = img_re
    img_ch2 = img_re

print("img_ch1 shape:", img_ch1.shape, "| img_ch2 shape:", img_ch2.shape)

# Choose lysosome channel
image = img_ch1  # or img_ch2, or (img_ch1 + img_ch2)

# =========================
# 2) SMOOTHING (lysosome channel)
# =========================
image_smooth = gaussian(image, sigma=1, preserve_range=True)

# =========================
# 3) BLOB DETECTION (lysosomes in 3D)
# =========================
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
# Convert sigma -> radius for 3D
blobs[:, 3] *= np.sqrt(3)
print(f"Detected {len(blobs)} lysosomes.")

# =========================
# 4) REGIONAL QUANTIFICATION (3D GRID)
# =========================
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)

region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx

# Unique ID and volume for each blob
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3)

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0],
    "y": blobs[:, 1],
    "x": blobs[:, 2],
    "diameter_voxels": blobs[:, 3] * 2,
    "radius_voxels": blobs[:, 3],
    "volume_voxels": volumes,
    "region_id": region_labels.astype(int)
}).round(3)

region_summary = (
    df.groupby("region_id")
      .agg(
          count=("id", "size"),
          avg_diameter=("diameter_voxels", "mean"),
          total_volume=("radius_voxels", lambda r: float(np.sum((4/3) * np.pi * (r**3))))
      )
      .reset_index()
      .round(3)
)

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# =========================
# 5) SEGMENT img_ch2 (cell bodies + axons)
# =========================
# --- TUNE THESE (voxel counts & thresholds) ---
min_object_voxels    = 500      # remove small speckles
min_cell_body_voxels = 20000    # components >= this are "cell bodies"
sato_sigmas          = (1, 2, 3)  # multiscale for axons (adjust to your data)
sato_percentile_thr  = 95        # fallback percentile for axon threshold if Otsu is too low#85

# Smooth + Otsu threshold for "all cells" mask
ch2_smooth = gaussian(img_ch2, sigma=1, preserve_range=True)
thr_all = filters.threshold_otsu(ch2_smooth)
mask_raw = ch2_smooth > thr_all

# 3D morphological cleanup
mask_clean = morphology.remove_small_objects(mask_raw, min_size=min_object_voxels)
mask_clean = morphology.binary_closing(mask_clean, morphology.ball(1))
mask_clean = ndi.binary_fill_holes(mask_clean)

# Label components and classify big ones as "cell bodies"
lbl = measure.label(mask_clean, connectivity=1)
props = measure.regionprops(lbl)
cell_body_labels = {p.label for p in props if p.area >= min_cell_body_voxels}
cell_bodies_mask = np.isin(lbl, list(cell_body_labels))
print(f"Segmentation: {len(props)} components; {len(cell_body_labels)} classified as cell bodies.")

# Axon detection with 3D Sato vesselness
sato_resp = sato(ch2_smooth.astype(np.float32), sigmas=sato_sigmas, black_ridges=False)
sato_norm = exposure.rescale_intensity(sato_resp, in_range="image", out_range=(0.0, 1.0))

# Threshold vesselness: try Otsu; if very low, use percentile fallback
thr_sato = filters.threshold_otsu(sato_norm)
if thr_sato <= 0.05:
    thr_sato = np.percentile(sato_norm, sato_percentile_thr)

axons_mask = (sato_norm > thr_sato)
# Constrain axons to inside the overall cellular mask and not cell bodies
axons_mask &= mask_clean & (~cell_bodies_mask)

# Optional: thin axons for display (comment in if desired)
# from skimage.morphology import skeletonize_3d
# axons_mask = skeletonize_3d(axons_mask)

# =========================
# 6) CLASSIFY LYSOSOMES BY OVERLAP (center-in-mask)
# =========================
def classify_point(zv, yv, xv):
    zi = int(round(zv)); yi = int(round(yv)); xi = int(round(xv))
    if (0 <= zi < cell_bodies_mask.shape[0] and
        0 <= yi < cell_bodies_mask.shape[1] and
        0 <= xi < cell_bodies_mask.shape[2]):
        if cell_bodies_mask[zi, yi, xi]:
            return "cell_body"
        elif axons_mask[zi, yi, xi]:
            return "axon"
        else:
            return "outside"
    return "outside"

compartment = np.array([classify_point(z, y, x) for z, y, x in blobs[:, :3]])
df["compartment"] = compartment

# =========================
# 7) SUMMARIES & EXPORTS (compartments)
# =========================
comp_counts = (
    df["compartment"].value_counts(dropna=False)
      .rename_axis("compartment")
      .reset_index(name="count")
)

region_comp_summary = (
    df.groupby(["region_id", "compartment"])
      .size()
      .reset_index(name="count")
      .pivot(index="region_id", columns="compartment", values="count")
      .fillna(0)
      .reset_index()
)

df.to_csv("lysosome_blobs_with_compartments.csv", index=False)
comp_counts.to_csv("lysosome_compartment_counts.csv", index=False)
region_comp_summary.to_csv("lysosome_region_compartment_summary.csv", index=False)

print("Saved:")
print("  - lysosome_blobs_with_compartments.csv")
print("  - lysosome_compartment_counts.csv")
print("  - lysosome_region_compartment_summary.csv")

# =========================
# 8) NAPARI VISUALIZATION
# =========================
viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')

# Lysosomes colored by region (original view)
properties_region = {'region': region_labels.astype(int)}
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties=properties_region,
    face_color='region',
    face_colormap='tab20',
    name='Detected Lysosomes'
)

# Region ID text labels at region centers
region_centers = []
region_ids = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)

viewer.add_points(
    region_centers,
    size=8,
    face_color='black',
    properties={"label": np.array(region_ids, dtype=int)},
    name='Region Labels',
    text={"string": "{label}", "size": 8, "color": "white", "anchor": "center"},
)

# Add segmentation masks as labels
viewer.add_labels(mask_clean.astype(np.uint8), name="Ch2: Cells (all)")
viewer.add_labels(cell_bodies_mask.astype(np.uint8), name="Ch2: Cell bodies")
viewer.add_labels(axons_mask.astype(np.uint8), name="Ch2: Axons")

# Lysosomes colored by compartment (cell_body / axon / outside)
comp_map = {"cell_body": 0, "axon": 1, "outside": 2}
comp_values = np.array([comp_map[c] for c in compartment], dtype=int)
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties={"compartment": comp_values.astype(float)},
    face_color="compartment",
    face_colormap="tab10",
    name="Lysosomes by compartment",
)

# =========================
# 9) VIDEO EXPORTS
# =========================
# Normalize safely to uint8
img_for_video = image
if img_for_video.dtype != np.uint8:
    rng = img_for_video.ptp()
    if rng == 0:
        img_norm = np.zeros_like(img_for_video, dtype=np.uint8)
    else:
        img_norm = ((img_for_video - img_for_video.min()) / rng * 255).astype(np.uint8)
else:
    img_norm = img_for_video

# Plain stack movie
frames = [img_norm[z] for z in range(img_norm.shape[0])]
try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except Exception:
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

# Overlay blobs per plane (yellow rings)
frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame_rgb = cv2.cvtColor(img_norm[z], cv2.COLOR_GRAY2BGR)
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
    for b in z_blobs:
        y_pix, x_pix = int(round(b[1])), int(round(b[2]))
        radius = max(1, int(round(b[3])))
        cv2.circle(frame_rgb, (x_pix, y_pix), radius, (0, 255, 255), 2)  # yellow
    frames_with_blobs.append(frame_rgb)

try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except Exception:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# =========================
# 10) START NAPARI
# =========================
napari.run()

In [ ]:
##################################Total-Variation (TV) denoise → Otsu (edge-preserving, good general default)############################################

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian
from skimage import filters, morphology, measure
from skimage.restoration import denoise_tv_chambolle
from scipy import ndimage as ndi
import napari
import pandas as pd
import imageio
import cv2

# =========================
# 0) INPUT
# =========================
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

# =========================
# 1) LOAD .CZI IMAGE (robust axis handling)
# =========================
with czifile.CziFile(file_path) as czi:
    arr = czi.asarray()
    axes_raw = czi.axes  # e.g., 'TCZYX' (can include singleton axes)

# Remove singleton axes from both data and axes string
squeezed = np.squeeze(arr)
axes = ''.join(ax for ax, sz in zip(axes_raw, arr.shape) if sz > 1)
img = squeezed
print("Raw CZI axes:", axes_raw, "Raw shape:", arr.shape)
print("Squeezed axes:", axes, "Squeezed shape:", img.shape)

def find_axis(axchar):
    return axes.find(axchar) if axchar in axes else None

# indices of important axes
c = find_axis('C')
z = find_axis('Z'); y = find_axis('Y'); x = find_axis('X')
if None in (z, y, x):
    raise RuntimeError(f"Expected Z/Y/X in axes, got '{axes}'")

# Reorder to (Z, Y, X) or (Z, C, Y, X) if C exists
img_re = np.moveaxis(img, [z, y, x], [0, 1, 2])  # Z,Y,X => 0,1,2
if c is not None:
    # Adjust channel index after moving Z/Y/X out
    c_adj = c - sum(idx is not None and idx < c for idx in (z, y, x))
    # Bring channel to axis=1 => shape (Z, C, Y, X)
    img_re = np.moveaxis(img_re, c_adj, 1)
    if img_re.shape[1] < 2:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 0]  # duplicate if only one channel
    else:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 1]
else:
    img_ch1 = img_re
    img_ch2 = img_re

print("img_ch1 shape:", img_ch1.shape, "| img_ch2 shape:", img_ch2.shape)

# Choose lysosome channel
image = img_ch1  # or img_ch2, or (img_ch1 + img_ch2)

# =========================
# 2) SMOOTHING (lysosome channel)
# =========================
image_smooth = gaussian(image, sigma=1, preserve_range=True)

# =========================
# 3) BLOB DETECTION (lysosomes in 3D)
# =========================
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
# Convert sigma -> radius for 3D
blobs[:, 3] *= np.sqrt(3)
print(f"Detected {len(blobs)} lysosomes.")

# =========================
# 4) REGIONAL QUANTIFICATION (3D GRID)
# =========================
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)

region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx

# Unique ID and volume for each blob
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3)

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0],
    "y": blobs[:, 1],
    "x": blobs[:, 2],
    "diameter_voxels": blobs[:, 3] * 2,
    "radius_voxels": blobs[:, 3],
    "volume_voxels": volumes,
    "region_id": region_labels.astype(int)
}).round(3)

region_summary = (
    df.groupby("region_id")
      .agg(
          count=("id", "size"),
          avg_diameter=("diameter_voxels", "mean"),
          total_volume=("radius_voxels", lambda r: float(np.sum((4/3) * np.pi * (r**3))))
      )
      .reset_index()
      .round(3)
)

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# =========================
# 5) SEGMENT img_ch2 (Option A: TV denoise → Otsu)
# =========================
# ---- TUNE THESE ----
min_object_voxels    = 500      # remove tiny speckles
min_cell_body_voxels = 60000    # components >= this => "cell bodies"#20000
tv_weight            = 0.01     # edge-preserving denoise strength#0.08

# Total-variation denoise (3D)
ch2_tv = denoise_tv_chambolle(img_ch2, weight=tv_weight, channel_axis=None)

# Global Otsu threshold on denoised volume
thr = filters.threshold_otsu(ch2_tv)
mask_raw = ch2_tv > thr

# 3D cleanup
#mask_clean = morphology.remove_small_objects(mask_raw, min_size=min_object_voxels)
mask_clean = morphology.binary_closing(mask_raw, morphology.ball(1))
mask_clean = ndi.binary_fill_holes(mask_clean)

# Label & classify large components as "cell bodies"
lbl = measure.label(mask_clean, connectivity=1)
props = measure.regionprops(lbl)
cell_body_labels = {p.label for p in props if p.area >= min_cell_body_voxels}
cell_bodies_mask = np.isin(lbl, list(cell_body_labels))

# Define axons as the rest of the cellular mask (inside mask_clean but not cell bodies)
axons_mask = mask_clean & (~cell_bodies_mask)

print(f"Segmentation (Option A): {len(props)} components; "
      f"{len(cell_body_labels)} classified as cell bodies.")

# =========================
# 6) CLASSIFY LYSOSOMES BY OVERLAP (center-in-mask)
# =========================
def classify_point(zv, yv, xv):
    zi = int(round(zv)); yi = int(round(yv)); xi = int(round(xv))
    if (0 <= zi < cell_bodies_mask.shape[0] and
        0 <= yi < cell_bodies_mask.shape[1] and
        0 <= xi < cell_bodies_mask.shape[2]):
        if cell_bodies_mask[zi, yi, xi]:
            return "cell_body"
        elif axons_mask[zi, yi, xi]:
            return "axon"
        else:
            return "outside"
    return "outside"

compartment = np.array([classify_point(z, y, x) for z, y, x in blobs[:, :3]])
df["compartment"] = compartment

# =========================
# 7) SUMMARIES & EXPORTS (compartments)
# =========================
comp_counts = (
    df["compartment"].value_counts(dropna=False)
      .rename_axis("compartment")
      .reset_index(name="count")
)

region_comp_summary = (
    df.groupby(["region_id", "compartment"])
      .size()
      .reset_index(name="count")
      .pivot(index="region_id", columns="compartment", values="count")
      .fillna(0)
      .reset_index()
)

df.to_csv("lysosome_blobs_with_compartments.csv", index=False)
comp_counts.to_csv("lysosome_compartment_counts.csv", index=False)
region_comp_summary.to_csv("lysosome_region_compartment_summary.csv", index=False)

print("Saved:")
print("  - lysosome_blobs_with_compartments.csv")
print("  - lysosome_compartment_counts.csv")
print("  - lysosome_region_compartment_summary.csv")

# =========================
# 8) NAPARI VISUALIZATION
# =========================
viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')

# Lysosomes colored by region (original view)
properties_region = {'region': region_labels.astype(int)}
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties=properties_region,
    face_color='region',
    face_colormap='tab20',
    name='Detected Lysosomes'
)

# Region ID text labels at region centers
region_centers = []
region_ids = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)

viewer.add_points(
    region_centers,
    size=8,
    face_color='black',
    properties={"label": np.array(region_ids, dtype=int)},
    name='Region Labels',
    text={"string": "{label}", "size": 8, "color": "white", "anchor": "center"},
)

# Add segmentation masks as labels
viewer.add_labels(mask_clean.astype(np.uint8), name="Ch2: Cells (all)")
viewer.add_labels(cell_bodies_mask.astype(np.uint8), name="Ch2: Cell bodies")
viewer.add_labels(axons_mask.astype(np.uint8), name="Ch2: Axons")

# Lysosomes colored by compartment (cell_body / axon / outside)
comp_map = {"cell_body": 0, "axon": 1, "outside": 2}
comp_values = np.array([comp_map[c] for c in compartment], dtype=int)
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties={"compartment": comp_values.astype(float)},
    face_color="compartment",
    face_colormap="tab10",
    name="Lysosomes by compartment",
)

# =========================
# 9) VIDEO EXPORTS
# =========================
# Normalize safely to uint8
img_for_video = image
if img_for_video.dtype != np.uint8:
    rng = img_for_video.ptp()
    if rng == 0:
        img_norm = np.zeros_like(img_for_video, dtype=np.uint8)
    else:
        img_norm = ((img_for_video - img_for_video.min()) / rng * 255).astype(np.uint8)
else:
    img_norm = img_for_video

# Plain stack movie
frames = [img_norm[z] for z in range(img_norm.shape[0])]
try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except Exception:
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

# Overlay blobs per plane (yellow rings)
frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame_rgb = cv2.cvtColor(img_norm[z], cv2.COLOR_GRAY2BGR)
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
    for b in z_blobs:
        y_pix, x_pix = int(round(b[1])), int(round(b[2]))
        radius = max(1, int(round(b[3])))
        cv2.circle(frame_rgb, (x_pix, y_pix), radius, (0, 255, 255), 2)  # yellow
    frames_with_blobs.append(frame_rgb)

try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except Exception:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# =========================
# 10) START NAPARI
# =========================
napari.run()

In [ ]:
#######################################Non-Local Means (NLM) → Sauvola local threshold (handles uneven illumination)###############3

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian
from skimage import filters, morphology, measure
from skimage.restoration import denoise_nl_means, estimate_sigma
from scipy import ndimage as ndi
import napari
import pandas as pd
import imageio
import cv2

# =========================
# 0) INPUT
# =========================
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

# =========================
# 1) LOAD .CZI IMAGE (robust axis handling)
# =========================
with czifile.CziFile(file_path) as czi:
    arr = czi.asarray()
    axes_raw = czi.axes  # e.g., 'TCZYX' (can include singleton axes)

# Remove singleton axes from both data and axes string
squeezed = np.squeeze(arr)
axes = ''.join(ax for ax, sz in zip(axes_raw, arr.shape) if sz > 1)
img = squeezed
print("Raw CZI axes:", axes_raw, "Raw shape:", arr.shape)
print("Squeezed axes:", axes, "Squeezed shape:", img.shape)

def find_axis(axchar):
    return axes.find(axchar) if axchar in axes else None

# indices of important axes
c = find_axis('C')
z = find_axis('Z'); y = find_axis('Y'); x = find_axis('X')
if None in (z, y, x):
    raise RuntimeError(f"Expected Z/Y/X in axes, got '{axes}'")

# Reorder to (Z, Y, X) or (Z, C, Y, X) if C exists
img_re = np.moveaxis(img, [z, y, x], [0, 1, 2])  # Z,Y,X => 0,1,2
if c is not None:
    # Adjust channel index after moving Z/Y/X out
    c_adj = c - sum(idx is not None and idx < c for idx in (z, y, x))
    # Bring channel to axis=1 => shape (Z, C, Y, X)
    img_re = np.moveaxis(img_re, c_adj, 1)
    if img_re.shape[1] < 2:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 0]  # duplicate if only one channel
    else:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 1]
else:
    img_ch1 = img_re
    img_ch2 = img_re

print("img_ch1 shape:", img_ch1.shape, "| img_ch2 shape:", img_ch2.shape)

# Choose lysosome channel
image = img_ch1  # or img_ch2, or (img_ch1 + img_ch2)

# =========================
# 2) SMOOTHING (lysosome channel)
# =========================
image_smooth = gaussian(image, sigma=1, preserve_range=True)

# =========================
# 3) BLOB DETECTION (lysosomes in 3D)
# =========================
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
# Convert sigma -> radius for 3D
blobs[:, 3] *= np.sqrt(3)
print(f"Detected {len(blobs)} lysosomes.")

# =========================
# 4) REGIONAL QUANTIFICATION (3D GRID)
# =========================
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)

region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx

# Unique ID and volume for each blob
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3)

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0],
    "y": blobs[:, 1],
    "x": blobs[:, 2],
    "diameter_voxels": blobs[:, 3] * 2,
    "radius_voxels": blobs[:, 3],
    "volume_voxels": volumes,
    "region_id": region_labels.astype(int)
}).round(3)

region_summary = (
    df.groupby("region_id")
      .agg(
          count=("id", "size"),
          avg_diameter=("diameter_voxels", "mean"),
          total_volume=("radius_voxels", lambda r: float(np.sum((4/3) * np.pi * (r**3))))
      )
      .reset_index()
      .round(3)
)

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# =========================
# 5) SEGMENT img_ch2 (Option B: NLM → per-slice Sauvola)
# =========================
# ---- TUNE THESE ----
min_object_voxels    = 500      # remove small speckles#500
min_cell_body_voxels = 20000    # components >= this => "cell bodies"#20000
sauvola_window       = 31       # odd; local window size in pixels
sauvola_k            = 0.2      # contrast parameter (0.1–0.3 typical)
nlm_patch_size       = 3        # 3x3x3 patches (applied slice-wise for Sauvola later)
nlm_patch_distance   = 5        # search distance
nlm_h_multiplier     = 1.15     # filter strength vs estimated sigma

# Normalize to [0,1] for stable denoising parameters
ch2_float = img_ch2.astype(np.float32)
maxval = ch2_float.max()
if maxval > 0:
    ch2_float /= maxval

# Non-Local Means denoising (3D volume)
sigma_est = float(np.mean(estimate_sigma(ch2_float, channel_axis=None)))
ch2_nlm = denoise_nl_means(
    ch2_float,
    patch_size=nlm_patch_size,
    patch_distance=nlm_patch_distance,
    h=nlm_h_multiplier * sigma_est,
    fast_mode=True,
    channel_axis=None
)

# Sauvola local threshold per Z-slice (handles uneven illumination)
local_mask = np.zeros_like(ch2_nlm, dtype=bool)
for zi in range(ch2_nlm.shape[0]):
    slice_f = ch2_nlm[zi].astype(np.float32)
    T = filters.threshold_sauvola(slice_f, window_size=sauvola_window, k=sauvola_k)
    local_mask[zi] = slice_f > T

# 3D cleanup
mask_clean = morphology.remove_small_objects(local_mask, min_size=min_object_voxels)
mask_clean = morphology.binary_closing(mask_clean, morphology.ball(1))
mask_clean = ndi.binary_fill_holes(mask_clean)

# Label components and classify big ones as "cell bodies"
lbl = measure.label(mask_clean, connectivity=1)
props = measure.regionprops(lbl)
cell_body_labels = {p.label for p in props if p.area >= min_cell_body_voxels}
cell_bodies_mask = np.isin(lbl, list(cell_body_labels))

# Define axons as the rest of the cellular mask
axons_mask = mask_clean & (~cell_bodies_mask)

print(f"Segmentation (Option B): {len(props)} components; "
      f"{len(cell_body_labels)} classified as cell bodies.")

# =========================
# 6) CLASSIFY LYSOSOMES BY OVERLAP (center-in-mask)
# =========================
def classify_point(zv, yv, xv):
    zi = int(round(zv)); yi = int(round(yv)); xi = int(round(xv))
    if (0 <= zi < cell_bodies_mask.shape[0] and
        0 <= yi < cell_bodies_mask.shape[1] and
        0 <= xi < cell_bodies_mask.shape[2]):
        if cell_bodies_mask[zi, yi, xi]:
            return "cell_body"
        elif axons_mask[zi, yi, xi]:
            return "axon"
        else:
            return "outside"
    return "outside"

compartment = np.array([classify_point(z, y, x) for z, y, x in blobs[:, :3]])
df["compartment"] = compartment

# =========================
# 7) SUMMARIES & EXPORTS (compartments)
# =========================
comp_counts = (
    df["compartment"].value_counts(dropna=False)
      .rename_axis("compartment")
      .reset_index(name="count")
)

region_comp_summary = (
    df.groupby(["region_id", "compartment"])
      .size()
      .reset_index(name="count")
      .pivot(index="region_id", columns="compartment", values="count")
      .fillna(0)
      .reset_index()
)

df.to_csv("lysosome_blobs_with_compartments.csv", index=False)
comp_counts.to_csv("lysosome_compartment_counts.csv", index=False)
region_comp_summary.to_csv("lysosome_region_compartment_summary.csv", index=False)

print("Saved:")
print("  - lysosome_blobs_with_compartments.csv")
print("  - lysosome_compartment_counts.csv")
print("  - lysosome_region_compartment_summary.csv")

# =========================
# 8) NAPARI VISUALIZATION
# =========================
viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')

# Lysosomes colored by region (original view)
properties_region = {'region': region_labels.astype(int)}
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties=properties_region,
    face_color='region',
    face_colormap='tab20',
    name='Detected Lysosomes'
)

# Region ID text labels at region centers
region_centers = []
region_ids = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)

viewer.add_points(
    region_centers,
    size=8,
    face_color='black',
    properties={"label": np.array(region_ids, dtype=int)},
    name='Region Labels',
    text={"string": "{label}", "size": 8, "color": "white", "anchor": "center"},
)

# Add segmentation masks as labels
viewer.add_labels(mask_clean.astype(np.uint8), name="Ch2: Cells (all)")
viewer.add_labels(cell_bodies_mask.astype(np.uint8), name="Ch2: Cell bodies")
viewer.add_labels(axons_mask.astype(np.uint8), name="Ch2: Axons")

# Lysosomes colored by compartment (cell_body / axon / outside)
comp_map = {"cell_body": 0, "axon": 1, "outside": 2}
comp_values = np.array([comp_map[c] for c in compartment], dtype=int)
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties={"compartment": comp_values.astype(float)},
    face_color="compartment",
    face_colormap="tab10",
    name="Lysosomes by compartment",
)

# =========================
# 9) VIDEO EXPORTS
# =========================
# Normalize safely to uint8
img_for_video = image
if img_for_video.dtype != np.uint8:
    rng = img_for_video.ptp()
    if rng == 0:
        img_norm = np.zeros_like(img_for_video, dtype=np.uint8)
    else:
        img_norm = ((img_for_video - img_for_video.min()) / rng * 255).astype(np.uint8)
else:
    img_norm = img_for_video

# Plain stack movie
frames = [img_norm[z] for z in range(img_norm.shape[0])]
try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except Exception:
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

# Overlay blobs per plane (yellow rings)
frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame_rgb = cv2.cvtColor(img_norm[z], cv2.COLOR_GRAY2BGR)
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
    for b in z_blobs:
        y_pix, x_pix = int(round(b[1])), int(round(b[2]))
        radius = max(1, int(round(b[3])))
        cv2.circle(frame_rgb, (x_pix, y_pix), radius, (0, 255, 255), 2)  # yellow
    frames_with_blobs.append(frame_rgb)

try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except Exception:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# =========================
# 10) START NAPARI
# =========================
napari.run()

In [ ]:
################################Option C: morphological white-tophat background correction → Yen global threshold#####################

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian
from skimage import filters, morphology, measure
from scipy import ndimage as ndi
import napari
import pandas as pd
import imageio
import cv2

# =========================
# 0) INPUT
# =========================
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

# =========================
# 1) LOAD .CZI IMAGE (robust axis handling)
# =========================
with czifile.CziFile(file_path) as czi:
    arr = czi.asarray()
    axes_raw = czi.axes  # e.g., 'TCZYX' (can include singleton axes)

# Remove singleton axes from both data and axes string
squeezed = np.squeeze(arr)
axes = ''.join(ax for ax, sz in zip(axes_raw, arr.shape) if sz > 1)
img = squeezed
print("Raw CZI axes:", axes_raw, "Raw shape:", arr.shape)
print("Squeezed axes:", axes, "Squeezed shape:", img.shape)

def find_axis(axchar):
    return axes.find(axchar) if axchar in axes else None

# indices of important axes
c = find_axis('C')
z = find_axis('Z'); y = find_axis('Y'); x = find_axis('X')
if None in (z, y, x):
    raise RuntimeError(f"Expected Z/Y/X in axes, got '{axes}'")

# Reorder to (Z, Y, X) or (Z, C, Y, X) if C exists
img_re = np.moveaxis(img, [z, y, x], [0, 1, 2])  # Z,Y,X => 0,1,2
if c is not None:
    # Adjust channel index after moving Z/Y/X out
    c_adj = c - sum(idx is not None and idx < c for idx in (z, y, x))
    # Bring channel to axis=1 => shape (Z, C, Y, X)
    img_re = np.moveaxis(img_re, c_adj, 1)
    if img_re.shape[1] < 2:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 0]  # duplicate if only one channel
    else:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 1]
else:
    img_ch1 = img_re
    img_ch2 = img_re

print("img_ch1 shape:", img_ch1.shape, "| img_ch2 shape:", img_ch2.shape)

# Choose lysosome channel
image = img_ch1  # or img_ch2, or (img_ch1 + img_ch2)

# =========================
# 2) SMOOTHING (lysosome channel)
# =========================
image_smooth = gaussian(image, sigma=1, preserve_range=True)

# =========================
# 3) BLOB DETECTION (lysosomes in 3D)
# =========================
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
# Convert sigma -> radius for 3D
blobs[:, 3] *= np.sqrt(3)
print(f"Detected {len(blobs)} lysosomes.")

# =========================
# 4) REGIONAL QUANTIFICATION (3D GRID)
# =========================
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)

region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx

# Unique ID and volume for each blob
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3)

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0],
    "y": blobs[:, 1],
    "x": blobs[:, 2],
    "diameter_voxels": blobs[:, 3] * 2,
    "radius_voxels": blobs[:, 3],
    "volume_voxels": volumes,
    "region_id": region_labels.astype(int)
}).round(3)

region_summary = (
    df.groupby("region_id")
      .agg(
          count=("id", "size"),
          avg_diameter=("diameter_voxels", "mean"),
          total_volume=("radius_voxels", lambda r: float(np.sum((4/3) * np.pi * (r**3))))
      )
      .reset_index()
      .round(3)
)

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# =========================
# 5) SEGMENT img_ch2 (Option C: white-tophat → Yen)
# =========================
# ---- TUNE THESE ----
min_object_voxels    = 500      # remove small speckles
min_cell_body_voxels = 20000    # components >= this => "cell bodies"
tophat_radius        = 3        # radius (voxels) for background removal

# Background correction with morphological white-tophat (3D)
bg_selem = morphology.ball(tophat_radius)  # increase if background varies slowly
ch2_corr = morphology.white_tophat(img_ch2, footprint=bg_selem)

# Global Yen threshold on corrected volume
thr = filters.threshold_yen(ch2_corr)
mask_raw = ch2_corr > thr

# 3D cleanup
mask_clean = morphology.remove_small_objects(mask_raw, min_size=min_object_voxels)
mask_clean = morphology.binary_closing(mask_clean, morphology.ball(1))
mask_clean = ndi.binary_fill_holes(mask_clean)

# Label components and classify big ones as "cell bodies"
lbl = measure.label(mask_clean, connectivity=1)
props = measure.regionprops(lbl)
cell_body_labels = {p.label for p in props if p.area >= min_cell_body_voxels}
cell_bodies_mask = np.isin(lbl, list(cell_body_labels))

# Define axons as the rest of the cellular mask
axons_mask = mask_clean & (~cell_bodies_mask)

print(f"Segmentation (Option C): {len(props)} components; "
      f"{len(cell_body_labels)} classified as cell bodies.")

# =========================
# 6) CLASSIFY LYSOSOMES BY OVERLAP (center-in-mask)
# =========================
def classify_point(zv, yv, xv):
    zi = int(round(zv)); yi = int(round(yv)); xi = int(round(xv))
    if (0 <= zi < cell_bodies_mask.shape[0] and
        0 <= yi < cell_bodies_mask.shape[1] and
        0 <= xi < cell_bodies_mask.shape[2]):
        if cell_bodies_mask[zi, yi, xi]:
            return "cell_body"
        elif axons_mask[zi, yi, xi]:
            return "axon"
        else:
            return "outside"
    return "outside"

compartment = np.array([classify_point(z, y, x) for z, y, x in blobs[:, :3]])
df["compartment"] = compartment

# =========================
# 7) SUMMARIES & EXPORTS (compartments)
# =========================
comp_counts = (
    df["compartment"].value_counts(dropna=False)
      .rename_axis("compartment")
      .reset_index(name="count")
)

region_comp_summary = (
    df.groupby(["region_id", "compartment"])
      .size()
      .reset_index(name="count")
      .pivot(index="region_id", columns="compartment", values="count")
      .fillna(0)
      .reset_index()
)

df.to_csv("lysosome_blobs_with_compartments.csv", index=False)
comp_counts.to_csv("lysosome_compartment_counts.csv", index=False)
region_comp_summary.to_csv("lysosome_region_compartment_summary.csv", index=False)

print("Saved:")
print("  - lysosome_blobs_with_compartments.csv")
print("  - lysosome_compartment_counts.csv")
print("  - lysosome_region_compartment_summary.csv")

# =========================
# 8) NAPARI VISUALIZATION
# =========================
viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')

# Lysosomes colored by region (original view)
properties_region = {'region': region_labels.astype(int)}
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties=properties_region,
    face_color='region',
    face_colormap='tab20',
    name='Detected Lysosomes'
)

# Region ID text labels at region centers
region_centers = []
region_ids = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)

viewer.add_points(
    region_centers,
    size=8,
    face_color='black',
    properties={"label": np.array(region_ids, dtype=int)},
    name='Region Labels',
    text={"string": "{label}", "size": 8, "color": "white", "anchor": "center"},
)

# Add segmentation masks as labels
viewer.add_labels(mask_clean.astype(np.uint8), name="Ch2: Cells (all)")
viewer.add_labels(cell_bodies_mask.astype(np.uint8), name="Ch2: Cell bodies")
viewer.add_labels(axons_mask.astype(np.uint8), name="Ch2: Axons")

# Lysosomes colored by compartment (cell_body / axon / outside)
comp_map = {"cell_body": 0, "axon": 1, "outside": 2}
comp_values = np.array([comp_map[c] for c in compartment], dtype=int)
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties={"compartment": comp_values.astype(float)},
    face_color="compartment",
    face_colormap="tab10",
    name="Lysosomes by compartment",
)

# =========================
# 9) VIDEO EXPORTS
# =========================
# Normalize safely to uint8
img_for_video = image
if img_for_video.dtype != np.uint8:
    rng = img_for_video.ptp()
    if rng == 0:
        img_norm = np.zeros_like(img_for_video, dtype=np.uint8)
    else:
        img_norm = ((img_for_video - img_for_video.min()) / rng * 255).astype(np.uint8)
else:
    img_norm = img_for_video

# Plain stack movie
frames = [img_norm[z] for z in range(img_norm.shape[0])]
try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except Exception:
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

# Overlay blobs per plane (yellow rings)
frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame_rgb = cv2.cvtColor(img_norm[z], cv2.COLOR_GRAY2BGR)
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
    for b in z_blobs:
        y_pix, x_pix = int(round(b[1])), int(round(b[2]))
        radius = max(1, int(round(b[3])))
        cv2.circle(frame_rgb, (x_pix, y_pix), radius, (0, 255, 255), 2)  # yellow
    frames_with_blobs.append(frame_rgb)

try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except Exception:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# =========================
# 10) START NAPARI
# =========================
napari.run()

In [ ]:
#####################################Option D: Vesselness (Frangi) for axons + cellular mask (white-tophat → Yen)#############################

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, frangi
from skimage import filters, morphology, measure
from scipy import ndimage as ndi
import napari
import pandas as pd
import imageio
import cv2

# =========================
# 0) INPUT
# =========================
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

# =========================
# 1) LOAD .CZI IMAGE (robust axis handling)
# =========================
with czifile.CziFile(file_path) as czi:
    arr = czi.asarray()
    axes_raw = czi.axes  # e.g., 'TCZYX' (can include singleton axes)

# Remove singleton axes from both data and axes string
squeezed = np.squeeze(arr)
axes = ''.join(ax for ax, sz in zip(axes_raw, arr.shape) if sz > 1)
img = squeezed
print("Raw CZI axes:", axes_raw, "Raw shape:", arr.shape)
print("Squeezed axes:", axes, "Squeezed shape:", img.shape)

def find_axis(axchar):
    return axes.find(axchar) if axchar in axes else None

# indices of important axes
c = find_axis('C')
z = find_axis('Z'); y = find_axis('Y'); x = find_axis('X')
if None in (z, y, x):
    raise RuntimeError(f"Expected Z/Y/X in axes, got '{axes}'")

# Reorder to (Z, Y, X) or (Z, C, Y, X) if C exists
img_re = np.moveaxis(img, [z, y, x], [0, 1, 2])  # Z,Y,X => 0,1,2
if c is not None:
    # Adjust channel index after moving Z/Y/X out
    c_adj = c - sum(idx is not None and idx < c for idx in (z, y, x))
    # Bring channel to axis=1 => shape (Z, C, Y, X)
    img_re = np.moveaxis(img_re, c_adj, 1)
    if img_re.shape[1] < 2:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 0]  # duplicate if only one channel
    else:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 1]
else:
    img_ch1 = img_re
    img_ch2 = img_re

print("img_ch1 shape:", img_ch1.shape, "| img_ch2 shape:", img_ch2.shape)

# Choose lysosome channel
image = img_ch1  # or img_ch2, or (img_ch1 + img_ch2)

# =========================
# 2) SMOOTHING (lysosome channel)
# =========================
image_smooth = gaussian(image, sigma=1, preserve_range=True)

# =========================
# 3) BLOB DETECTION (lysosomes in 3D)
# =========================
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
# Convert sigma -> radius for 3D
blobs[:, 3] *= np.sqrt(3)
print(f"Detected {len(blobs)} lysosomes.")

# =========================
# 4) REGIONAL QUANTIFICATION (3D GRID)
# =========================
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)

region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx

# Unique ID and volume for each blob
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3)

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0],
    "y": blobs[:, 1],
    "x": blobs[:, 2],
    "diameter_voxels": blobs[:, 3] * 2,
    "radius_voxels": blobs[:, 3],
    "volume_voxels": volumes,
    "region_id": region_labels.astype(int)
}).round(3)

region_summary = (
    df.groupby("region_id")
      .agg(
          count=("id", "size"),
          avg_diameter=("diameter_voxels", "mean"),
          total_volume=("radius_voxels", lambda r: float(np.sum((4/3) * np.pi * (r**3))))
      )
      .reset_index()
      .round(3)
)

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# =========================
# 5) SEGMENT img_ch2 (Option D: Vesselness for axons + cellular mask)
# =========================
# ---- TUNE THESE ----
min_object_voxels    = 10000      # remove small speckles#500
min_cell_body_voxels = 100000    # components >= this => "cell bodies"#20000
tophat_radius        = 3        # background correction radius (voxels)
frangi_sigmas        = (1, 2, 3)  # multiscale for axons; widen for thicker neurites
frangi_alpha_beta    = (0.5, 0.5)
frangi_gamma         = 15
vessel_thr_min       = 0.01     # fallback minimum threshold for vesselness#0.05

# --- A) Cellular mask (white-tophat → Yen) ---
bg_selem = morphology.ball(tophat_radius)
ch2_corr = morphology.white_tophat(img_ch2, footprint=bg_selem)
thr_cell = filters.threshold_yen(ch2_corr)  # try Otsu if Yen is too aggressive
mask_raw = ch2_corr > thr_cell

# 3D cleanup
mask_clean = morphology.remove_small_objects(mask_raw, min_size=min_object_voxels)
mask_clean = morphology.binary_closing(mask_clean, morphology.ball(1))
mask_clean = ndi.binary_fill_holes(mask_clean)

# Label & classify large components as "cell bodies"
lbl = measure.label(mask_clean, connectivity=1)
props = measure.regionprops(lbl)
cell_body_labels = {p.label for p in props if p.area >= min_cell_body_voxels}
cell_bodies_mask = np.isin(lbl, list(cell_body_labels))

print(f"Cell mask: {len(props)} components; {len(cell_body_labels)} classified as cell bodies.")

# --- B) Axon vesselness (Frangi) ---
ch2_f32 = img_ch2.astype(np.float32)
vess = frangi(
    ch2_f32,
    sigmas=frangi_sigmas,
    alpha=frangi_alpha_beta[0],
    beta=frangi_alpha_beta[1],
    gamma=frangi_gamma,
    black_ridges=False,  # bright axons on dark background
)
# Threshold vesselness: Otsu, fallback to min if too low
thr_v = filters.threshold_otsu(vess)
if thr_v < vessel_thr_min:
    thr_v = vessel_thr_min

axons_mask = (vess > thr_v)
# Constrain axons to inside cell mask and exclude cell bodies
axons_mask &= mask_clean & (~cell_bodies_mask)

# Optional: thin axons for display (comment-in if you want)
# from skimage.morphology import skeletonize_3d
# axons_mask = skeletonize_3d(axons_mask)

# =========================
# 6) CLASSIFY LYSOSOMES BY OVERLAP (center-in-mask)
# =========================
def classify_point(zv, yv, xv):
    zi = int(round(zv)); yi = int(round(yv)); xi = int(round(xv))
    if (0 <= zi < cell_bodies_mask.shape[0] and
        0 <= yi < cell_bodies_mask.shape[1] and
        0 <= xi < cell_bodies_mask.shape[2]):
        if cell_bodies_mask[zi, yi, xi]:
            return "cell_body"
        elif axons_mask[zi, yi, xi]:
            return "axon"
        else:
            return "outside"
    return "outside"

compartment = np.array([classify_point(z, y, x) for z, y, x in blobs[:, :3]])
df["compartment"] = compartment

# =========================
# 7) SUMMARIES & EXPORTS (compartments)
# =========================
comp_counts = (
    df["compartment"].value_counts(dropna=False)
      .rename_axis("compartment")
      .reset_index(name="count")
)

region_comp_summary = (
    df.groupby(["region_id", "compartment"])
      .size()
      .reset_index(name="count")
      .pivot(index="region_id", columns="compartment", values="count")
      .fillna(0)
      .reset_index()
)

df.to_csv("lysosome_blobs_with_compartments.csv", index=False)
comp_counts.to_csv("lysosome_compartment_counts.csv", index=False)
region_comp_summary.to_csv("lysosome_region_compartment_summary.csv", index=False)

print("Saved:")
print("  - lysosome_blobs_with_compartments.csv")
print("  - lysosome_compartment_counts.csv")
print("  - lysosome_region_compartment_summary.csv")

# =========================
# 8) NAPARI VISUALIZATION
# =========================
viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')

# Lysosomes colored by region (original view)
properties_region = {'region': region_labels.astype(int)}
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties=properties_region,
    face_color='region',
    face_colormap='tab20',
    name='Detected Lysosomes'
)

# Region ID text labels at region centers
region_centers = []
region_ids = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)

viewer.add_points(
    region_centers,
    size=8,
    face_color='black',
    properties={"label": np.array(region_ids, dtype=int)},
    name='Region Labels',
    text={"string": "{label}", "size": 8, "color": "white", "anchor": "center"},
)

# Add segmentation masks as labels
viewer.add_labels(mask_clean.astype(np.uint8), name="Ch2: Cells (all)")
viewer.add_labels(cell_bodies_mask.astype(np.uint8), name="Ch2: Cell bodies")
viewer.add_labels(axons_mask.astype(np.uint8), name="Ch2: Axons")

# Lysosomes colored by compartment (cell_body / axon / outside)
comp_map = {"cell_body": 0, "axon": 1, "outside": 2}
comp_values = np.array([comp_map[c] for c in compartment], dtype=int)
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties={"compartment": comp_values.astype(float)},
    face_color="compartment",
    face_colormap="tab10",
    name="Lysosomes by compartment",
)

# =========================
# 9) VIDEO EXPORTS
# =========================
# Normalize safely to uint8
img_for_video = image
if img_for_video.dtype != np.uint8:
    rng = img_for_video.ptp()
    if rng == 0:
        img_norm = np.zeros_like(img_for_video, dtype=np.uint8)
    else:
        img_norm = ((img_for_video - img_for_video.min()) / rng * 255).astype(np.uint8)
else:
    img_norm = img_for_video

# Plain stack movie
frames = [img_norm[z] for z in range(img_norm.shape[0])]
try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except Exception:
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

# Overlay blobs per plane (yellow rings)
frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame_rgb = cv2.cvtColor(img_norm[z], cv2.COLOR_GRAY2BGR)
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
    for b in z_blobs:
        y_pix, x_pix = int(round(b[1])), int(round(b[2]))
        radius = max(1, int(round(b[3])))
        cv2.circle(frame_rgb, (x_pix, y_pix), radius, (0, 255, 255), 2)  # yellow
    frames_with_blobs.append(frame_rgb)

try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except Exception:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# =========================
# 10) START NAPARI
# =========================
napari.run()

In [ ]:
######################################Option E: watershed on distance transform#############################################3333

In [ ]:
#NO FUNCIONA
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian
from skimage import filters, morphology, measure
from skimage.restoration import denoise_tv_chambolle
from skimage.segmentation import watershed
from scipy import ndimage as ndi
import napari
import pandas as pd
import imageio
import cv2

# =========================
# 0) INPUT
# =========================
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

# =========================
# 1) LOAD .CZI IMAGE (robust axis handling)
# =========================
with czifile.CziFile(file_path) as czi:
    arr = czi.asarray()
    axes_raw = czi.axes  # e.g., 'TCZYX'

squeezed = np.squeeze(arr)
axes = ''.join(ax for ax, sz in zip(axes_raw, arr.shape) if sz > 1)
img = squeezed
print("Raw CZI axes:", axes_raw, "Raw shape:", arr.shape)
print("Squeezed axes:", axes, "Squeezed shape:", img.shape)

def find_axis(axchar):
    return axes.find(axchar) if axchar in axes else None

c = find_axis('C')
z = find_axis('Z'); y = find_axis('Y'); x = find_axis('X')
if None in (z, y, x):
    raise RuntimeError(f"Expected Z/Y/X in axes, got '{axes}'")

# Reorder to (Z, Y, X) or (Z, C, Y, X) if C exists
img_re = np.moveaxis(img, [z, y, x], [0, 1, 2])
if c is not None:
    c_adj = c - sum(idx is not None and idx < c for idx in (z, y, x))
    img_re = np.moveaxis(img_re, c_adj, 1)  # (Z, C, Y, X)
    if img_re.shape[1] < 2:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 0]
    else:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 1]
else:
    img_ch1 = img_re
    img_ch2 = img_re

print("img_ch1 shape:", img_ch1.shape, "| img_ch2 shape:", img_ch2.shape)

# Choose lysosome channel
image = img_ch1  # or img_ch2, or (img_ch1 + img_ch2)

# =========================
# 2) SMOOTHING (lysosome channel)
# =========================
image_smooth = gaussian(image, sigma=1, preserve_range=True)

# =========================
# 3) BLOB DETECTION (lysosomes in 3D)
# =========================
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
blobs[:, 3] *= np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# =========================
# 4) REGIONAL QUANTIFICATION (3D GRID)
# =========================
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)

region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3)

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0],
    "y": blobs[:, 1],
    "x": blobs[:, 2],
    "diameter_voxels": blobs[:, 3] * 2,
    "radius_voxels": blobs[:, 3],
    "volume_voxels": volumes,
    "region_id": region_labels.astype(int)
}).round(3)

region_summary = (
    df.groupby("region_id")
      .agg(
          count=("id", "size"),
          avg_diameter=("diameter_voxels", "mean"),
          total_volume=("radius_voxels", lambda r: float(np.sum((4/3)*np.pi*(r**3))))
      )
      .reset_index()
      .round(3)
)

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# =========================
# 5) SEGMENT img_ch2 (Option E: Watershed on distance transform)
# =========================
# ---- TUNE THESE ----
min_object_voxels    = 500      # remove tiny speckles
min_cell_body_voxels = 20000    # labels >= this => "cell bodies"
tv_weight            = 0.08     # TV denoise before threshold
peak_footprint_rad   = 5        # enlarge for fewer seeds; shrink for more

# --- A) Foreground mask (TV denoise → Otsu) ---
ch2_tv = denoise_tv_chambolle(img_ch2, weight=tv_weight, channel_axis=None)
thr = filters.threshold_otsu(ch2_tv)
mask_raw = ch2_tv > thr

# cleanup
mask_clean = morphology.remove_small_objects(mask_raw, min_size=min_object_voxels)
mask_clean = morphology.binary_closing(mask_clean, morphology.ball(1))
mask_clean = ndi.binary_fill_holes(mask_clean)

# --- B) Watershed to separate touching somas ---
dist = ndi.distance_transform_edt(mask_clean)

# seed maxima
foot = morphology.ball(peak_footprint_rad)
seeds_bool = morphology.local_maxima(dist, footprint=foot)
markers = measure.label(seeds_bool)

# Fallback if no seeds were found
if markers.max() == 0:
    max_pos = np.unravel_index(np.argmax(dist), dist.shape)
    seeds_fallback = np.zeros_like(dist, dtype=bool)
    seeds_fallback[max_pos] = True
    markers = measure.label(seeds_fallback)

labels_ws = watershed(-dist, markers, mask=mask_clean)

# Keep large labels as cell bodies
props_ws = measure.regionprops(labels_ws)
cell_body_labels = {p.label for p in props_ws if p.area >= min_cell_body_voxels}
cell_bodies_mask = np.isin(labels_ws, list(cell_body_labels))

# Axons = cellular mask minus cell bodies
axons_mask = mask_clean & (~cell_bodies_mask)

print(f"Watershed: {len(props_ws)} labels; {len(cell_body_labels)} cell bodies ≥ {min_cell_body_voxels} voxels.")

# =========================
# 6) CLASSIFY LYSOSOMES BY OVERLAP (center-in-mask)
# =========================
def classify_point(zv, yv, xv):
    zi = int(round(zv)); yi = int(round(yv)); xi = int(round(xv))
    if (0 <= zi < cell_bodies_mask.shape[0] and
        0 <= yi < cell_bodies_mask.shape[1] and
        0 <= xi < cell_bodies_mask.shape[2]):
        if cell_bodies_mask[zi, yi, xi]:
            return "cell_body"
        elif axons_mask[zi, yi, xi]:
            return "axon"
        else:
            return "outside"
    return "outside"

compartment = np.array([classify_point(z, y, x) for z, y, x in blobs[:, :3]])
df["compartment"] = compartment

# =========================
# 7) SUMMARIES & EXPORTS (compartments)
# =========================
comp_counts = (
    df["compartment"].value_counts(dropna=False)
      .rename_axis("compartment")
      .reset_index(name="count")
)

region_comp_summary = (
    df.groupby(["region_id", "compartment"])
      .size()
      .reset_index(name="count")
      .pivot(index="region_id", columns="compartment", values="count")
      .fillna(0)
      .reset_index()
)

df.to_csv("lysosome_blobs_with_compartments.csv", index=False)
comp_counts.to_csv("lysosome_compartment_counts.csv", index=False)
region_comp_summary.to_csv("lysosome_region_compartment_summary.csv", index=False)

print("Saved:")
print("  - lysosome_blobs_with_compartments.csv")
print("  - lysosome_compartment_counts.csv")
print("  - lysosome_region_compartment_summary.csv")

# =========================
# 8) NAPARI VISUALIZATION
# =========================
viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')

# Lysosomes colored by region (original view)
properties_region = {'region': region_labels.astype(int)}
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties=properties_region,
    face_color='region',
    face_colormap='tab20',
    name='Detected Lysosomes'
)

# Region ID text labels at region centers
region_centers = []
region_ids = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)

viewer.add_points(
    region_centers,
    size=8,
    face_color='black',
    properties={"label": np.array(region_ids, dtype=int)},
    name='Region Labels',
    text={"string": "{label}", "size": 8, "color": "white", "anchor": "center"},
)

# Add segmentation masks as labels
viewer.add_labels(mask_clean.astype(np.uint8), name="Ch2: Cells (all)")
viewer.add_labels(cell_bodies_mask.astype(np.uint8), name="Ch2: Cell bodies")
viewer.add_labels(axons_mask.astype(np.uint8), name="Ch2: Axons")

# Lysosomes colored by compartment (cell_body / axon / outside)
comp_map = {"cell_body": 0, "axon": 1, "outside": 2}
comp_values = np.array([comp_map[c] for c in compartment], dtype=int)
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties={"compartment": comp_values.astype(float)},
    face_color="compartment",
    face_colormap="tab10",
    name="Lysosomes by compartment",
)

# =========================
# 9) VIDEO EXPORTS
# =========================
img_for_video = image
if img_for_video.dtype != np.uint8:
    rng = img_for_video.ptp()
    if rng == 0:
        img_norm = np.zeros_like(img_for_video, dtype=np.uint8)
    else:
        img_norm = ((img_for_video - img_for_video.min()) / rng * 255).astype(np.uint8)
else:
    img_norm = img_for_video

frames = [img_norm[z] for z in range(img_norm.shape[0])]
try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except Exception:
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame_rgb = cv2.cvtColor(img_norm[z], cv2.COLOR_GRAY2BGR)
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
    for b in z_blobs:
        y_pix, x_pix = int(round(b[1])), int(round(b[2]))
        radius = max(1, int(round(b[3])))
        cv2.circle(frame_rgb, (x_pix, y_pix), radius, (0, 255, 255), 2)
    frames_with_blobs.append(frame_rgb)

try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except Exception:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# =========================
# 10) START NAPARI
# =========================
napari.run()

In [ ]:
###########################################Option F: Random-walker segmentation#####################################################

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian
from skimage import filters, morphology, measure
from skimage.segmentation import random_walker
from scipy import ndimage as ndi
import napari
import pandas as pd
import imageio
import cv2

# =========================
# 0) INPUT
# =========================
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

# =========================
# 1) LOAD .CZI IMAGE (robust axis handling)
# =========================
with czifile.CziFile(file_path) as czi:
    arr = czi.asarray()
    axes_raw = czi.axes  # e.g., 'TCZYX' (can include singleton axes)

# Remove singleton axes from both data and axes string
squeezed = np.squeeze(arr)
axes = ''.join(ax for ax, sz in zip(axes_raw, arr.shape) if sz > 1)
img = squeezed
print("Raw CZI axes:", axes_raw, "Raw shape:", arr.shape)
print("Squeezed axes:", axes, "Squeezed shape:", img.shape)

def find_axis(axchar):
    return axes.find(axchar) if axchar in axes else None

# indices of important axes
c = find_axis('C')
z = find_axis('Z'); y = find_axis('Y'); x = find_axis('X')
if None in (z, y, x):
    raise RuntimeError(f"Expected Z/Y/X in axes, got '{axes}'")

# Reorder to (Z, Y, X) or (Z, C, Y, X) if C exists
img_re = np.moveaxis(img, [z, y, x], [0, 1, 2])  # Z,Y,X => 0,1,2
if c is not None:
    # Adjust channel index after moving Z/Y/X out
    c_adj = c - sum(idx is not None and idx < c for idx in (z, y, x))
    # Bring channel to axis=1 => shape (Z, C, Y, X)
    img_re = np.moveaxis(img_re, c_adj, 1)
    if img_re.shape[1] < 2:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 0]  # duplicate if only one channel
    else:
        img_ch1 = img_re[:, 0]
        img_ch2 = img_re[:, 1]
else:
    img_ch1 = img_re
    img_ch2 = img_re

print("img_ch1 shape:", img_ch1.shape, "| img_ch2 shape:", img_ch2.shape)

# Choose lysosome channel
image = img_ch1  # or img_ch2, or (img_ch1 + img_ch2)

# =========================
# 2) SMOOTHING (lysosome channel)
# =========================
image_smooth = gaussian(image, sigma=1, preserve_range=True)

# =========================
# 3) BLOB DETECTION (lysosomes in 3D)
# =========================
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
# Convert sigma -> radius for 3D
blobs[:, 3] *= np.sqrt(3)
print(f"Detected {len(blobs)} lysosomes.")

# =========================
# 4) REGIONAL QUANTIFICATION (3D GRID)
# =========================
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)

region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx

# Unique ID and volume for each blob
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3)

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0],
    "y": blobs[:, 1],
    "x": blobs[:, 2],
    "diameter_voxels": blobs[:, 3] * 2,
    "radius_voxels": blobs[:, 3],
    "volume_voxels": volumes,
    "region_id": region_labels.astype(int)
}).round(3)

region_summary = (
    df.groupby("region_id")
      .agg(
          count=("id", "size"),
          avg_diameter=("diameter_voxels", "mean"),
          total_volume=("radius_voxels", lambda r: float(np.sum((4/3) * np.pi * (r**3))))
      )
      .reset_index()
      .round(3)
)

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# =========================
# 5) SEGMENT img_ch2 (Option F: Random-walker)
# =========================
# ---- TUNE THESE ----
min_object_voxels    = 500      # remove small speckles
min_cell_body_voxels = 20000    # components >= this => "cell bodies"
rw_low_high_percent  = (20, 80) # percentiles for BG/FG seeds
rw_beta              = 50       # smoothing strength (higher = smoother)
rw_mode              = 'cg'     # 'cg' works without pyamg; 'cg_mg' is faster if pyamg installed

# Scale to [0,1] for RW stability
ch2_float = img_ch2.astype(np.float32)
vmax = ch2_float.max()
if vmax > 0:
    ch2_float /= vmax

# Build conservative markers from intensity percentiles
low, high = np.percentile(ch2_float, rw_low_high_percent)
bg_seed = ch2_float < low
fg_seed = ch2_float > high

# Optional: tighten seeds to avoid leakage (erode FG, dilate BG slightly)
fg_seed = morphology.binary_erosion(fg_seed, morphology.ball(1))
bg_seed = morphology.binary_dilation(bg_seed, morphology.ball(1))

markers = np.zeros_like(ch2_float, dtype=np.uint8)
markers[bg_seed] = 1  # background label
markers[fg_seed] = 2  # foreground (cells)

# Random-walker segmentation
rw_labels = random_walker(ch2_float, markers, beta=rw_beta, mode=rw_mode)
mask_raw = (rw_labels == 2)

# 3D cleanup
mask_clean = morphology.remove_small_objects(mask_raw, min_size=min_object_voxels)
mask_clean = morphology.binary_closing(mask_clean, morphology.ball(1))
mask_clean = ndi.binary_fill_holes(mask_clean)

# Label components and classify big ones as "cell bodies"
lbl = measure.label(mask_clean, connectivity=1)
props = measure.regionprops(lbl)
cell_body_labels = {p.label for p in props if p.area >= min_cell_body_voxels}
cell_bodies_mask = np.isin(lbl, list(cell_body_labels))

# Define axons as the rest of the cellular mask
axons_mask = mask_clean & (~cell_bodies_mask)

print(f"Segmentation (Option F): {len(props)} components; "
      f"{len(cell_body_labels)} classified as cell bodies.")

# =========================
# 6) CLASSIFY LYSOSOMES BY OVERLAP (center-in-mask)
# =========================
def classify_point(zv, yv, xv):
    zi = int(round(zv)); yi = int(round(yv)); xi = int(round(xv))
    if (0 <= zi < cell_bodies_mask.shape[0] and
        0 <= yi < cell_bodies_mask.shape[1] and
        0 <= xi < cell_bodies_mask.shape[2]):
        if cell_bodies_mask[zi, yi, xi]:
            return "cell_body"
        elif axons_mask[zi, yi, xi]:
            return "axon"
        else:
            return "outside"
    return "outside"

compartment = np.array([classify_point(z, y, x) for z, y, x in blobs[:, :3]])
df["compartment"] = compartment

# =========================
# 7) SUMMARIES & EXPORTS (compartments)
# =========================
comp_counts = (
    df["compartment"].value_counts(dropna=False)
      .rename_axis("compartment")
      .reset_index(name="count")
)

region_comp_summary = (
    df.groupby(["region_id", "compartment"])
      .size()
      .reset_index(name="count")
      .pivot(index="region_id", columns="compartment", values="count")
      .fillna(0)
      .reset_index()
)

df.to_csv("lysosome_blobs_with_compartments.csv", index=False)
comp_counts.to_csv("lysosome_compartment_counts.csv", index=False)
region_comp_summary.to_csv("lysosome_region_compartment_summary.csv", index=False)

print("Saved:")
print("  - lysosome_blobs_with_compartments.csv")
print("  - lysosome_compartment_counts.csv")
print("  - lysosome_region_compartment_summary.csv")

# =========================
# 8) NAPARI VISUALIZATION
# =========================
viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')

# Lysosomes colored by region (original view)
properties_region = {'region': region_labels.astype(int)}
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties=properties_region,
    face_color='region',
    face_colormap='tab20',
    name='Detected Lysosomes'
)

# Region ID text labels at region centers
region_centers = []
region_ids = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)

viewer.add_points(
    region_centers,
    size=8,
    face_color='black',
    properties={"label": np.array(region_ids, dtype=int)},
    name='Region Labels',
    text={"string": "{label}", "size": 8, "color": "white", "anchor": "center"},
)

# Add segmentation masks as labels
viewer.add_labels(mask_clean.astype(np.uint8), name="Ch2: Cells (all)")
viewer.add_labels(cell_bodies_mask.astype(np.uint8), name="Ch2: Cell bodies")
viewer.add_labels(axons_mask.astype(np.uint8), name="Ch2: Axons")

# Lysosomes colored by compartment (cell_body / axon / outside)
comp_map = {"cell_body": 0, "axon": 1, "outside": 2}
comp_values = np.array([comp_map[c] for c in compartment], dtype=int)
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties={"compartment": comp_values.astype(float)},
    face_color="compartment",
    face_colormap="tab10",
    name="Lysosomes by compartment",
)

# =========================
# 9) VIDEO EXPORTS
# =========================
# Normalize safely to uint8
img_for_video = image
if img_for_video.dtype != np.uint8:
    rng = img_for_video.ptp()
    if rng == 0:
        img_norm = np.zeros_like(img_for_video, dtype=np.uint8)
    else:
        img_norm = ((img_for_video - img_for_video.min()) / rng * 255).astype(np.uint8)
else:
    img_norm = img_for_video

# Plain stack movie
frames = [img_norm[z] for z in range(img_norm.shape[0])]
try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except Exception:
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

# Overlay blobs per plane (yellow rings)
frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame_rgb = cv2.cvtColor(img_norm[z], cv2.COLOR_GRAY2BGR)
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
    for b in z_blobs:
        y_pix, x_pix = int(round(b[1])), int(round(b[2]))
        radius = max(1, int(round(b[3])))
        cv2.circle(frame_rgb, (x_pix, y_pix), radius, (0, 255, 255), 2)  # yellow
    frames_with_blobs.append(frame_rgb)

try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except Exception:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# =========================
# 10) START NAPARI
# =========================
napari.run()

In [ ]:
#########################isotropic resampling + Sato + hysteresis############################

In [ ]:
# %%
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian
import napari
import pandas as pd
import imageio
import cv2

from skimage.filters import sato, threshold_yen, apply_hysteresis_threshold
from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_closing, ball, skeletonize_3d, white_tophat
)
from skimage.transform import resize
import xml.etree.ElementTree as ET


# --------------------------
# Helpers
# --------------------------
def parse_czi_spacing_um(meta_xml: str):
    """
    Try to parse voxel spacing (X,Y,Z) in micrometers from CZI metadata XML.
    Returns (dz_um, dy_um, dx_um) or None if parsing fails.
    """
    try:
        root = ET.fromstring(meta_xml)
        spacings = {}
        # Many CZI files store scaling under .../Scaling/Items/Distance[@Id]
        for dist in root.findall(".//Scaling//Items//Distance"):
            axis = dist.get("Id") or dist.findtext("Id")
            val = dist.findtext("Value")
            unit = dist.findtext("Unit")
            if axis and val:
                v = float(val)
                if unit:
                    u = unit.strip().lower()
                else:
                    u = "m"  # assume meters if missing

                if u in {"m", "meter", "metre"}:
                    v_um = v * 1e6
                elif u in {"µm", "um", "micrometer", "micrometre"}:
                    v_um = v
                else:
                    # Unknown unit; assume meters
                    v_um = v * 1e6
                spacings[axis.upper()] = v_um

        dz_um = spacings.get("Z", None)
        dy_um = spacings.get("Y", None)
        dx_um = spacings.get("X", None)
        if dz_um and dy_um and dx_um:
            return dz_um, dy_um, dx_um
    except Exception:
        pass
    return None


def safe_minmax_norm(volume):
    vol = volume.astype(np.float32)
    vmin, vmax = float(vol.min()), float(vol.max())
    if vmax > vmin:
        vol = (vol - vmin) / (vmax - vmin)
    else:
        vol[:] = 0.0
    return vol


def to_uint8_stack(volume):
    vol = safe_minmax_norm(volume)
    return (vol * 255).astype(np.uint8)


# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs axon channel (as in your original)
image = img_ch1        # lysosome channel
image_2 = img_ch2      # axon/soma channel

# --------------------------
# 2) SMOOTHING (CH1)
# --------------------------
image_smooth = gaussian(image, sigma=1)

# --------------------------
# 3) BLOB DETECTION (CH1)
# --------------------------
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
# convert sigma->radius for LoG blobs (3D)
blobs[:, 3] = blobs[:, 3] * np.sqrt(3)

print(f"Detected {len(blobs)} lysosomes.")

# --------------------------
# 4) REGIONAL QUANTIFICATION (CH1) on a 3D GRID
# --------------------------
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

z_idx = np.digitize(blobs[:, 0], z_bins) - 1
y_idx = np.digitize(blobs[:, 1], y_bins) - 1
x_idx = np.digitize(blobs[:, 2], x_bins) - 1

region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx

# --- Calculate unique ID and volume for each blob ---
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3)

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0],
    "y": blobs[:, 1],
    "x": blobs[:, 2],
    "diameter_voxels": blobs[:, 3] * 2,
    "radius_voxels": blobs[:, 3],
    "volume_voxels": volumes,
    "region_id": region_labels
})

region_summary = df.groupby("region_id").agg(
    count=("radius_voxels", "size"),
    avg_diameter=("diameter_voxels", "mean"),
    total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
).reset_index()

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 5) NAPARI VISUALIZATION (CH1)
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')

# Color blobs by region for visual grouping
properties = {'region': region_labels.astype(int)}
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties=properties,
    face_color='region',
    face_colormap='tab20',
    name='Detected Lysosomes'
)

# ----Add region ID text labels at region centers (for CH1 grid)----
region_centers = []
region_ids = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)
region_ids = np.array(region_ids)

viewer.add_points(
    region_centers,
    size=8,
    face_color='black',
    name='Region Labels',
    text=[str(r) for r in region_ids]
)

# --------------------------
# 3B) AXON SEGMENTATION — OPTION 1 (Isotropic + Sato + Hysteresis)
# --------------------------

# 3B-1. Determine voxel spacing (µm)
spacing_um = parse_czi_spacing_um(meta_xml)
if spacing_um is None:
    # Fallback if metadata parsing fails — please replace with your actual spacings
    dz_um, dy_um, dx_um = 0.5, 0.1, 0.1
    print("WARNING: Could not parse voxel size from CZI metadata. "
          f"Using fallback spacings (µm): dz={dz_um}, dy={dy_um}, dx={dx_um}")
else:
    dz_um, dy_um, dx_um = spacing_um
    print(f"Parsed voxel spacings (µm): dz={dz_um:.4f}, dy={dy_um:.4f}, dx={dx_um:.4f}")

# 3B-2. Resample to isotropic voxels
target_um = min(dz_um, dy_um, dx_um)
scale_z = dz_um / target_um
scale_y = dy_um / target_um
scale_x = dx_um / target_um

ch2 = rescale_intensity(image_2.astype(np.float32), in_range='image', out_range=(0, 1))
new_shape = (
    int(round(ch2.shape[0] * scale_z)),
    int(round(ch2.shape[1] * scale_y)),
    int(round(ch2.shape[2] * scale_x)),
)
print("Resampling to isotropic grid:", new_shape, "target voxel:", target_um, "µm")

# NOTE: linear interpolation for image; binary masks use order=0 later
ch2_iso = resize(ch2, new_shape, order=1, anti_aliasing=True, preserve_range=True)

# 3B-3. Light background correction (white top-hat)
ch2_iso = white_tophat(ch2_iso, footprint=ball(2))

# 3B-4. Multi-scale Sato tubeness
# Choose radii in micrometers that match your neurites/soma "tube-like" parts.
# Convert to sigma in (isotropic) voxels. The 0.7 factor works well empirically.
r_min_um, r_max_um = 0.2, 2.5
sigmas = np.linspace(r_min_um/target_um * 0.7, r_max_um/target_um * 0.7, 10)

resp = sato(ch2_iso, sigmas=sigmas, black_ridges=False)

# 3B-5. Hysteresis thresholding (robust to weak links)
hi = threshold_yen(resp)
lo = 0.40 * hi
axon_mask_iso = apply_hysteresis_threshold(resp, lo, hi)

# 3B-6. Cleanup
axon_mask_iso = remove_small_objects(axon_mask_iso, min_size=500, connectivity=3)
axon_mask_iso = binary_closing(axon_mask_iso, ball(1))

# 3B-7. Back to original grid
axon_mask = resize(
    axon_mask_iso.astype(np.uint8),
    image_2.shape,
    order=0,  # nearest for labels
    preserve_range=True
).astype(bool)

# 3B-8. Skeleton (optional)
axon_skel = skeletonize_3d(axon_mask)

# --------------------------
# 4) REGIONAL QUANTIFICATION (CH2) on a 3D GRID
# --------------------------
num_bins_2 = (4, 4, 4)
z_bins_2 = np.linspace(0, image_2.shape[0], num_bins_2[0] + 1, dtype=int)
y_bins_2 = np.linspace(0, image_2.shape[1], num_bins_2[1] + 1, dtype=int)
x_bins_2 = np.linspace(0, image_2.shape[2], num_bins_2[2] + 1, dtype=int)

region_records = []
for zi in range(num_bins_2[0]):
    for yi in range(num_bins_2[1]):
        for xi in range(num_bins_2[2]):
            z0, z1 = z_bins_2[zi], z_bins_2[zi+1]
            y0, y1 = y_bins_2[yi], y_bins_2[yi+1]
            x0, x1 = x_bins_2[xi], x_bins_2[xi+1]
            rid = zi * (num_bins_2[1] * num_bins_2[2]) + yi * num_bins_2[2] + xi

            sub_mask  = axon_mask[z0:z1, y0:y1, x0:x1]
            sub_skel  = axon_skel[z0:z1, y0:y1, x0:x1]
            vox_total = sub_mask.size
            vox_axon  = int(sub_mask.sum())
            vox_skel  = int(sub_skel.sum())  # skeleton voxels ~ length proxy (in voxel units)

            region_records.append({
                "region_id": rid,
                "axon_voxels": vox_axon,
                "region_voxels": vox_total,
                "axon_volume_fraction": vox_axon / vox_total if vox_total > 0 else 0.0,
                "skeleton_voxels": vox_skel
            })

df_axon_regions = pd.DataFrame(region_records)

# --------------------------
# 4B) COLOCALIZATION: ch1 blobs inside axon mask (CH2)
# --------------------------
def _in_bounds(z, y, x, shape):
    return (0 <= z < shape[0]) and (0 <= y < shape[1]) and (0 <= x < shape[2])

in_axon = []
for zc, yc, xc in blobs[:, :3]:
    zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
    if _in_bounds(zz, yy, xx, axon_mask.shape):
        in_axon.append(bool(axon_mask[zz, yy, xx]))
    else:
        in_axon.append(False)

df["in_axon_mask_ch2"] = in_axon

# per-region colocalization rates
coloc_summary = df.groupby("region_id").agg(
    blobs_total=("id", "size"),
    blobs_in_axon=("in_axon_mask_ch2", "sum")
).reset_index()
coloc_summary["frac_blobs_in_axon"] = (
    coloc_summary["blobs_in_axon"] / coloc_summary["blobs_total"].replace(0, np.nan)
)

# --------------------------
# 4C) SAVE CHANNEL-2 SUMMARIES
# --------------------------
df_axon_regions.to_csv("axon_regions_ch2.csv", index=False)
coloc_summary.to_csv("blobs_in_axon_by_region.csv", index=False)
print("Saved: axon_regions_ch2.csv and blobs_in_axon_by_region.csv")

# --------------------------
# 5B) NAPARI VISUALIZATION (CH2)
# --------------------------
viewer_2 = napari.Viewer()
viewer_2.add_image(image_2, name='Channel 2 (raw)')
viewer_2.add_labels(axon_mask.astype(np.uint8), name='Axon mask (Sato, isotropic)')
viewer_2.add_image(axon_skel.astype(np.uint8), name='Axon skeleton', blending='additive', contrast_limits=(0,1))

# --------------------------
# 6) MOVIES / OVERLAYS
# --------------------------
# CH1 video (raw)
img_norm = to_uint8_stack(image)
frames = [img_norm[z] for z in range(img_norm.shape[0])]

try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except TypeError:
    print('MP4 export failed, trying GIF...')
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

# CH1 video with blobs overlay
frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame = img_norm[z].copy()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    # Draw blobs at this z-slice
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
    for b in z_blobs:
        y, x = int(round(b[1])), int(round(b[2]))
        radius = int(round(b[3]))
        cv2.circle(frame_rgb, (x, y), radius, (0, 255, 255), 2)  # yellow
    frames_with_blobs.append(frame_rgb)

try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except TypeError:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# CH2 base stack (raw)
img_norm_2 = to_uint8_stack(image_2)
frames_2 = [img_norm_2[z2] for z2 in range(img_norm_2.shape[0])]
try:
    imageio.mimsave('axon_stack_ch2.mp4', frames_2, fps=8, format='FFMPEG')
    print("Saved: axon_stack_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_stack_ch2.gif', frames_2, fps=8)
    print("Saved: axon_stack_ch2.gif")

# CH2 overlay with axon mask (colored)
frames_mask_overlay = []
for z2 in range(img_norm_2.shape[0]):
    base = cv2.cvtColor(img_norm_2[z2], cv2.COLOR_GRAY2BGR)
    mask = (axon_mask[z2].astype(np.uint8) * 255)
    color_mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(base, 1.0, color_mask, 0.35, 0.0)
    frames_mask_overlay.append(overlay)

try:
    imageio.mimsave('axon_mask_overlay_ch2.mp4', frames_mask_overlay, fps=8, format='FFMPEG')
    print("Saved: axon_mask_overlay_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_mask_overlay_ch2.gif', frames_mask_overlay, fps=8)
    print("Saved: axon_mask_overlay_ch2.gif")

# --------------------------
# 7) RUN VIEWERS
# --------------------------
napari.run()

In [ ]:
###############################Meijering ridges + slice-wise local thresholds####################################

In [ ]:
# %%
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, meijering, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, skeletonize_3d
)

# --------------------------
# Helpers
# --------------------------
def safe_minmax_norm(volume):
    vol = volume.astype(np.float32)
    vmin, vmax = float(vol.min()), float(vol.max())
    if vmax > vmin:
        vol = (vol - vmin) / (vmax - vmin)
    else:
        vol[:] = 0.0
    return vol

def to_uint8_stack(volume):
    vol = safe_minmax_norm(volume)
    return (vol * 255).astype(np.uint8)

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs axon channel (as in your original)
image = img_ch1        # lysosome channel
image_2 = img_ch2      # axon/soma channel

# --------------------------
# 2) SMOOTHING (CH1)
# --------------------------
image_smooth = gaussian(image, sigma=1)

# --------------------------
# 3) BLOB DETECTION (CH1)
# --------------------------
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
# convert sigma->radius for LoG blobs (3D)
blobs[:, 3] = blobs[:, 3] * np.sqrt(3)

print(f"Detected {len(blobs)} lysosomes.")

# --------------------------
# 4) REGIONAL QUANTIFICATION (CH1) on a 3D GRID
# --------------------------
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

z_idx = np.digitize(blobs[:, 0], z_bins) - 1
y_idx = np.digitize(blobs[:, 1], y_bins) - 1
x_idx = np.digitize(blobs[:, 2], x_bins) - 1

region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx

# --- Calculate unique ID and volume for each blob ---
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3)

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0],
    "y": blobs[:, 1],
    "x": blobs[:, 2],
    "diameter_voxels": blobs[:, 3] * 2,
    "radius_voxels": blobs[:, 3],
    "volume_voxels": volumes,
    "region_id": region_labels
})

region_summary = df.groupby("region_id").agg(
    count=("radius_voxels", "size"),
    avg_diameter=("diameter_voxels", "mean"),
    total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
).reset_index()

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 5) NAPARI VISUALIZATION (CH1)
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')

# Color blobs by region for visual grouping
properties = {'region': region_labels.astype(int)}
viewer.add_points(
    blobs[:, :3],
    size=blobs[:, 3] * 2,
    properties=properties,
    face_color='region',
    face_colormap='tab20',
    name='Detected Lysosomes'
)

# ----Add region ID text labels at region centers (for CH1 grid)----
region_centers = []
region_ids = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)
region_ids = np.array(region_ids)

viewer.add_points(
    region_centers,
    size=8,
    face_color='black',
    name='Region Labels',
    text=[str(r) for r in region_ids]
)

# --------------------------
# 3B) AXON SEGMENTATION — OPTION 2 (Meijering + local thresholds)
# --------------------------
# Normalize CH2 into [0,1] and denoise lightly
ch2 = rescale_intensity(image_2.astype(np.float32), in_range='image', out_range=(0,1))
ch2 = gaussian(ch2, sigma=0.6, preserve_range=True)

# Meijering (bright ridges → black_ridges=False)
resp = meijering(ch2, sigmas=np.linspace(0.6, 2.0, 8), black_ridges=False)

# Local threshold per z-slice (handles depth-varying background)
axon_mask = np.zeros_like(resp, dtype=bool)
for z in range(resp.shape[0]):
    R = resp[z]
    # block_size must be odd; tweak offset factor (e.g., -0.3 to -0.7)*std for your data
    t = threshold_local(R, block_size=11, offset=-0.3*np.std(R))#51#0.5
    axon_mask[z] = R > t

# Cleanup (remove speckles; open/close small gaps)
axon_mask = remove_small_objects(axon_mask, min_size=4000, connectivity=3)#400
axon_mask = binary_opening(axon_mask, ball(1))
axon_mask = binary_closing(axon_mask, ball(1))

# (Optional) SKELETON for geometry/length
axon_skel = skeletonize_3d(axon_mask)

# --------------------------
# 4) REGIONAL QUANTIFICATION (CH2) on a 3D GRID
# --------------------------
num_bins_2 = (4, 4, 4)
z_bins_2 = np.linspace(0, image_2.shape[0], num_bins_2[0] + 1, dtype=int)
y_bins_2 = np.linspace(0, image_2.shape[1], num_bins_2[1] + 1, dtype=int)
x_bins_2 = np.linspace(0, image_2.shape[2], num_bins_2[2] + 1, dtype=int)

region_records = []
for zi in range(num_bins_2[0]):
    for yi in range(num_bins_2[1]):
        for xi in range(num_bins_2[2]):
            z0, z1 = z_bins_2[zi], z_bins_2[zi+1]
            y0, y1 = y_bins_2[yi], y_bins_2[yi+1]
            x0, x1 = x_bins_2[xi], x_bins_2[xi+1]
            rid = zi * (num_bins_2[1] * num_bins_2[2]) + yi * num_bins_2[2] + xi

            sub_mask  = axon_mask[z0:z1, y0:y1, x0:x1]
            sub_skel  = axon_skel[z0:z1, y0:y1, x0:x1]
            vox_total = sub_mask.size
            vox_axon  = int(sub_mask.sum())
            vox_skel  = int(sub_skel.sum())  # skeleton voxels ~ length proxy (in voxel units)

            region_records.append({
                "region_id": rid,
                "axon_voxels": vox_axon,
                "region_voxels": vox_total,
                "axon_volume_fraction": vox_axon / vox_total if vox_total > 0 else 0.0,
                "skeleton_voxels": vox_skel
            })

df_axon_regions = pd.DataFrame(region_records)

# --------------------------
# 4B) COLOCALIZATION: ch1 blobs inside axon mask (CH2)
# --------------------------
def _in_bounds(z, y, x, shape):
    return (0 <= z < shape[0]) and (0 <= y < shape[1]) and (0 <= x < shape[2])

in_axon = []
for zc, yc, xc in blobs[:, :3]:
    zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
    if _in_bounds(zz, yy, xx, axon_mask.shape):
        in_axon.append(bool(axon_mask[zz, yy, xx]))
    else:
        in_axon.append(False)

df["in_axon_mask_ch2"] = in_axon

# per-region colocalization rates
coloc_summary = df.groupby("region_id").agg(
    blobs_total=("id", "size"),
    blobs_in_axon=("in_axon_mask_ch2", "sum")
).reset_index()
coloc_summary["frac_blobs_in_axon"] = (
    coloc_summary["blobs_in_axon"] / coloc_summary["blobs_total"].replace(0, np.nan)
)

# --------------------------
# 4C) SAVE CHANNEL-2 SUMMARIES
# --------------------------
df_axon_regions.to_csv("axon_regions_ch2.csv", index=False)
coloc_summary.to_csv("blobs_in_axon_by_region.csv", index=False)
print("Saved: axon_regions_ch2.csv and blobs_in_axon_by_region.csv")

# --------------------------
# 5B) NAPARI VISUALIZATION (CH2)
# --------------------------
viewer_2 = napari.Viewer()
viewer_2.add_image(image_2, name='Channel 2 (raw)')
viewer_2.add_labels(axon_mask.astype(np.uint8), name='Axon mask (Meijering + local)')
viewer_2.add_image(axon_skel.astype(np.uint8), name='Axon skeleton', blending='additive', contrast_limits=(0,1))

# --------------------------
# 6) MOVIES / OVERLAYS
# --------------------------
# CH1 video (raw)
img_norm = to_uint8_stack(image)
frames = [img_norm[z] for z in range(img_norm.shape[0])]

try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except TypeError:
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

# CH1 video with blobs overlay
frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame = img_norm[z].copy()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    # Draw blobs at this z-slice
    z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
    for b in z_blobs:
        y, x = int(round(b[1])), int(round(b[2]))
        radius = int(round(b[3]))
        cv2.circle(frame_rgb, (x, y), radius, (0, 255, 255), 2)  # yellow
    frames_with_blobs.append(frame_rgb)

try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except TypeError:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# CH2 base stack (raw)
img_norm_2 = to_uint8_stack(image_2)
frames_2 = [img_norm_2[z2] for z2 in range(img_norm_2.shape[0])]
try:
    imageio.mimsave('axon_stack_ch2.mp4', frames_2, fps=8, format='FFMPEG')
    print("Saved: axon_stack_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_stack_ch2.gif', frames_2, fps=8)
    print("Saved: axon_stack_ch2.gif")

# CH2 overlay with axon mask (colored)
frames_mask_overlay = []
for z2 in range(img_norm_2.shape[0]):
    base = cv2.cvtColor(img_norm_2[z2], cv2.COLOR_GRAY2BGR)
    mask = (axon_mask[z2].astype(np.uint8) * 255)
    color_mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(base, 1.0, color_mask, 0.35, 0.0)
    frames_mask_overlay.append(overlay)

try:
    imageio.mimsave('axon_mask_overlay_ch2.mp4', frames_mask_overlay, fps=8, format='FFMPEG')
    print("Saved: axon_mask_overlay_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_mask_overlay_ch2.gif', frames_mask_overlay, fps=8)
    print("Saved: axon_mask_overlay_ch2.gif")

# --------------------------
# 7) RUN VIEWERS
# --------------------------
napari.run()

In [ ]:
##################################Soma + axon fusion: blobness + ridges#############################3

In [ ]:
# %%
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, sato, threshold_yen, apply_hysteresis_threshold
from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_closing, binary_opening, binary_dilation, ball,
    skeletonize_3d
)
import napari
import pandas as pd
import imageio
import cv2


# --------------------------
# Helpers
# --------------------------
def safe_minmax_norm(volume):
    vol = volume.astype(np.float32)
    vmin, vmax = float(vol.min()), float(vol.max())
    if vmax > vmin:
        vol = (vol - vmin) / (vmax - vmin)
    else:
        vol[:] = 0.0
    return vol

def to_uint8_stack(volume):
    vol = safe_minmax_norm(volume)
    return (vol * 255).astype(np.uint8)


# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs axon channel (as in your original)
image = img_ch1        # lysosome channel
image_2 = img_ch2      # axon/soma channel


# --------------------------
# 2) SMOOTHING (CH1)
# --------------------------
image_smooth = gaussian(image, sigma=1, preserve_range=True)


# --------------------------
# 3) BLOB DETECTION (CH1)
# --------------------------
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)
# convert sigma->radius for LoG blobs (3D)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)
print(f"Detected {len(blobs)} lysosomes.")


# --------------------------
# 4) REGIONAL QUANTIFICATION (CH1) on a 3D GRID
# --------------------------
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.digitize(blobs[:, 0], z_bins) - 1
    y_idx = np.digitize(blobs[:, 1], y_bins) - 1
    x_idx = np.digitize(blobs[:, 2], x_bins) - 1
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([])

# --- Calculate unique ID and volume for each blob ---
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})
if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")


# --------------------------
# 5) NAPARI VISUALIZATION (CH1)
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')

# Color blobs by region for visual grouping
if len(blobs) > 0:
    properties = {'region': region_labels.astype(int)}
    viewer.add_points(
        blobs[:, :3],
        size=blobs[:, 3] * 2,
        properties=properties,
        face_color='region',
        face_colormap='tab20',
        name='Detected Lysosomes'
    )

# ----Add region ID text labels at region centers (for CH1 grid)----
region_centers = []
region_ids = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)
region_ids = np.array(region_ids)

viewer.add_points(
    region_centers,
    size=8,
    face_color='black',
    name='Region Labels',
    text=[str(r) for r in region_ids]
)


# ================================================================
# 3B) AXON/ soma SEGMENTATION — OPTION 3 (Blobness for soma + ridges for axon)
# ================================================================
# Normalize CH2 into [0,1] and denoise lightly
ch2 = rescale_intensity(image_2.astype(np.float32), in_range='image', out_range=(0,1))
ch2 = gaussian(ch2, sigma=0.6, preserve_range=True)

# ---- Soma detection (blobness + bright-core threshold)
# 1) High-intensity core via Yen
yen_thr = threshold_yen(ch2)
soma_core = ch2 > yen_thr   # bright cores (soma typically among brightest)

# 2) Blob LoG seeds, then dilate to cover soma more completely
#    Tune min_sigma/max_sigma to expected soma radius (in *voxels*)
#    Example: soma radius ~ 3–8 vox → min_sigma~2, max_sigma~8
blobs_soma = blob_log(
    ch2,
    min_sigma=2.0,
    max_sigma=8.0,
    num_sigma=10,
    threshold=0.02,      # raise to reduce false positives, lower for sensitivity
    overlap=0.5
)
soma_mask = soma_core.copy()
if len(blobs_soma) > 0:
    # blobs_soma columns: (z, y, x, sigma)
    # Convert LoG sigma to radius approximation (3D)
    blobs_soma[:, 3] = blobs_soma[:, 3] * np.sqrt(3)
    seed = np.zeros_like(soma_mask, dtype=bool)
    for zc, yc, xc, s in blobs_soma:
        rr = int(max(3, round(1.5 * s)))   # expand ~1.5× radius
        zc, yc, xc = int(round(zc)), int(round(yc)), int(round(xc))
        z0, z1 = max(0, zc-rr), min(ch2.shape[0], zc+rr+1)
        y0, y1 = max(0, yc-rr), min(ch2.shape[1], yc+rr+1)
        x0, x1 = max(0, xc-rr), min(ch2.shape[2], xc+rr+1)
        seed[z0:z1, y0:y1, x0:x1] = True
    # Merge core + seeds, then dilate to smooth edges
    soma_mask = (soma_mask | seed)
    soma_mask = binary_dilation(soma_mask, ball(2))

# ---- Axon detection (ridges via Sato tubeness)
# Choose sigmas in *voxels* that match axon radii (e.g., 0.6–3.0)
resp = sato(ch2, sigmas=np.linspace(0.8, 2.5, 10), black_ridges=False)

# Robust thresholding with hysteresis to keep weak links connected to strong ones
hi = threshold_yen(resp)
lo = 0.6 * hi
axon_mask = apply_hysteresis_threshold(resp, lo, hi)

# ---- Merge soma and axon, then clean up
mask = (soma_mask | axon_mask)
mask = remove_small_objects(mask, min_size=600, connectivity=3)
mask = binary_opening(mask, ball(1))
mask = binary_closing(mask, ball(1))

# (Optional) Skeleton for geometry/length
axon_skel = skeletonize_3d(mask)


# --------------------------
# 4) REGIONAL QUANTIFICATION (CH2) on a 3D GRID
# --------------------------
num_bins_2 = (4, 4, 4)
z_bins_2 = np.linspace(0, image_2.shape[0], num_bins_2[0] + 1, dtype=int)
y_bins_2 = np.linspace(0, image_2.shape[1], num_bins_2[1] + 1, dtype=int)
x_bins_2 = np.linspace(0, image_2.shape[2], num_bins_2[2] + 1, dtype=int)

region_records = []
for zi in range(num_bins_2[0]):
    for yi in range(num_bins_2[1]):
        for xi in range(num_bins_2[2]):
            z0, z1 = z_bins_2[zi], z_bins_2[zi+1]
            y0, y1 = y_bins_2[yi], y_bins_2[yi+1]
            x0, x1 = x_bins_2[xi], x_bins_2[xi+1]
            rid = zi * (num_bins_2[1] * num_bins_2[2]) + yi * num_bins_2[2] + xi

            sub_mask  = mask[z0:z1, y0:y1, x0:x1]
            sub_skel  = axon_skel[z0:z1, y0:y1, x0:x1]
            vox_total = sub_mask.size
            vox_axon  = int(sub_mask.sum())
            vox_skel  = int(sub_skel.sum())  # skeleton voxels ~ length proxy (in voxel units)

            region_records.append({
                "region_id": rid,
                "axon_voxels": vox_axon,
                "region_voxels": vox_total,
                "axon_volume_fraction": vox_axon / vox_total if vox_total > 0 else 0.0,
                "skeleton_voxels": vox_skel
            })

df_axon_regions = pd.DataFrame(region_records)

# --------------------------
# 4B) COLOCALIZATION: ch1 blobs inside (soma+axon) mask (CH2)
# --------------------------
def _in_bounds(z, y, x, shape):
    return (0 <= z < shape[0]) and (0 <= y < shape[1]) and (0 <= x < shape[2])

in_axon = []
for zc, yc, xc in (blobs[:, :3] if len(blobs) > 0 else []):
    zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
    if _in_bounds(zz, yy, xx, mask.shape):
        in_axon.append(bool(mask[zz, yy, xx]))
    else:
        in_axon.append(False)

if len(blobs) > 0:
    df["in_axon_mask_ch2"] = in_axon
    # per-region colocalization rates
    coloc_summary = df.groupby("region_id").agg(
        blobs_total=("id", "size"),
        blobs_in_axon=("in_axon_mask_ch2", "sum")
    ).reset_index()
    coloc_summary["frac_blobs_in_axon"] = (
        coloc_summary["blobs_in_axon"] / coloc_summary["blobs_total"].replace(0, np.nan)
    )
else:
    coloc_summary = pd.DataFrame(columns=["region_id", "blobs_total", "blobs_in_axon", "frac_blobs_in_axon"])

# --------------------------
# 4C) SAVE CHANNEL-2 SUMMARIES
# --------------------------
df_axon_regions.to_csv("axon_regions_ch2.csv", index=False)
coloc_summary.to_csv("blobs_in_axon_by_region.csv", index=False)
print("Saved: axon_regions_ch2.csv and blobs_in_axon_by_region.csv")


# --------------------------
# 5B) NAPARI VISUALIZATION (CH2)
# --------------------------
viewer_2 = napari.Viewer()
viewer_2.add_image(image_2, name='Channel 2 (raw)')
viewer_2.add_labels(mask.astype(np.uint8), name='Soma+Axon mask (blob+ridges)')
viewer_2.add_image(axon_skel.astype(np.uint8), name='Skeleton', blending='additive', contrast_limits=(0,1))


# --------------------------
# 6) MOVIES / OVERLAYS
# --------------------------
# CH1 video (raw)
img_norm = to_uint8_stack(image)
frames = [img_norm[z] for z in range(img_norm.shape[0])]

try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except TypeError:
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

# CH1 video with blobs overlay
frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame = img_norm[z].copy()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    # Draw blobs at this z-slice
    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            radius = int(round(b[3]))
            cv2.circle(frame_rgb, (x, y), radius, (0, 255, 255), 2)  # yellow
    frames_with_blobs.append(frame_rgb)

try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except TypeError:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# CH2 base stack (raw)
img_norm_2 = to_uint8_stack(image_2)
frames_2 = [img_norm_2[z2] for z2 in range(img_norm_2.shape[0])]
try:
    imageio.mimsave('axon_stack_ch2.mp4', frames_2, fps=8, format='FFMPEG')
    print("Saved: axon_stack_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_stack_ch2.gif', frames_2, fps=8)
    print("Saved: axon_stack_ch2.gif")

# CH2 overlay with final mask (colored)
frames_mask_overlay = []
for z2 in range(img_norm_2.shape[0]):
    base = cv2.cvtColor(img_norm_2[z2], cv2.COLOR_GRAY2BGR)
    mask_u8 = (mask[z2].astype(np.uint8) * 255)
    color_mask = cv2.applyColorMap(mask_u8, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(base, 1.0, color_mask, 0.35, 0.0)
    frames_mask_overlay.append(overlay)

try:
    imageio.mimsave('axon_mask_overlay_ch2.mp4', frames_mask_overlay, fps=8, format='FFMPEG')
    print("Saved: axon_mask_overlay_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_mask_overlay_ch2.gif', frames_mask_overlay, fps=8)
    print("Saved: axon_mask_overlay_ch2.gif")


# --------------------------
# 7) RUN VIEWERS
# --------------------------
napari.run()

In [ ]:
##########################Quick wins for your existing Frangi approach##############################################33

In [ ]:
# %%
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, frangi, threshold_yen, apply_hysteresis_threshold
from skimage.exposure import rescale_intensity
from skimage.morphology import (
    remove_small_objects, binary_closing, ball, skeletonize_3d
)
import napari
import pandas as pd
import imageio
import cv2


# --------------------------
# Helpers
# --------------------------
def safe_minmax_norm(volume):
    vol = volume.astype(np.float32)
    vmin, vmax = float(vol.min()), float(vol.max())
    if vmax > vmin:
        vol = (vol - vmin) / (vmax - vmin)
    else:
        vol[:] = 0.0
    return vol

def to_uint8_stack(volume):
    vol = safe_minmax_norm(volume)
    return (vol * 255).astype(np.uint8)


# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs axon channel (as in your original)
image  = img_ch1       # lysosome channel
image_2 = img_ch2      # axon/soma channel


# --------------------------
# 2) SMOOTHING (CH1)
# --------------------------
image_smooth = gaussian(image, sigma=1, preserve_range=True)


# --------------------------
# 3) BLOB DETECTION (CH1)
# --------------------------
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.005,
    overlap=0.5
)

# convert sigma->radius for LoG blobs (3D)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)

print(f"Detected {len(blobs)} lysosomes.")


# --------------------------
# 4) REGIONAL QUANTIFICATION (CH1) on a 3D GRID
# --------------------------
num_bins = (4, 4, 4)  # (z_bins, y_bins, x_bins)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.digitize(blobs[:, 0], z_bins) - 1
    y_idx = np.digitize(blobs[:, 1], y_bins) - 1
    x_idx = np.digitize(blobs[:, 2], x_bins) - 1
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([])

# --- Calculate unique ID and volume for each blob ---
blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("radius_voxels", lambda r: np.sum(4/3 * np.pi * r**3))
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")


# --------------------------
# 5) NAPARI VISUALIZATION (CH1)
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image, name='Lysosome Channel')

# Color blobs by region for visual grouping
if len(blobs) > 0:
    properties = {'region': region_labels.astype(int)}
    viewer.add_points(
        blobs[:, :3],
        size=blobs[:, 3] * 2,
        properties=properties,
        face_color='region',
        face_colormap='tab20',
        name='Detected Lysosomes'
    )

# ----Add region ID text labels at region centers (for CH1 grid)----
region_centers = []
region_ids = []
for zi in range(num_bins[0]):
    for yi in range(num_bins[1]):
        for xi in range(num_bins[2]):
            cz = (z_bins[zi] + z_bins[zi+1]) / 2
            cy = (y_bins[yi] + y_bins[yi+1]) / 2
            cx = (x_bins[xi] + x_bins[xi+1]) / 2
            region_centers.append([cz, cy, cx])
            region_ids.append(zi * num_bins[1] * num_bins[2] + yi * num_bins[2] + xi)
region_centers = np.array(region_centers)
region_ids = np.array(region_ids)

viewer.add_points(
    region_centers,
    size=8,
    face_color='black',
    name='Region Labels',
    text=[str(r) for r in region_ids]
)


# ================================================================
# 3B) AXON SEGMENTATION — QUICK FRANGI IMPROVEMENTS
# ================================================================
# Normalize CH2 into [0,1] for stable filtering
ch2 = rescale_intensity(image_2.astype(np.float32), in_range='image', out_range=(0,1))

# Frangi with tuned parameters (bright-on-dark → black_ridges=False)
resp = frangi(
    ch2,
    sigmas=np.linspace(0.8, 3.0, 12),  # tune to neurite radius (voxels)
    alpha=0.5, beta=0.5, gamma=15,
    black_ridges=False
)

# Robust hysteresis thresholding (better continuity than a single global cut)
hi = threshold_yen(resp)
lo = 0.35 * hi
axon_mask = apply_hysteresis_threshold(resp, lo, hi)

# Cleanup (remove speckles; close tiny gaps)
axon_mask = remove_small_objects(axon_mask, min_size=500, connectivity=3)
axon_mask = binary_closing(axon_mask, ball(1))

# Optional: skeleton for geometry/length
axon_skel = skeletonize_3d(axon_mask)


# --------------------------
# 4) REGIONAL QUANTIFICATION (CH2) on a 3D GRID
# --------------------------
num_bins_2 = (4, 4, 4)
z_bins_2 = np.linspace(0, image_2.shape[0], num_bins_2[0] + 1, dtype=int)
y_bins_2 = np.linspace(0, image_2.shape[1], num_bins_2[1] + 1, dtype=int)
x_bins_2 = np.linspace(0, image_2.shape[2], num_bins_2[2] + 1, dtype=int)

region_records = []
for zi in range(num_bins_2[0]):
    for yi in range(num_bins_2[1]):
        for xi in range(num_bins_2[2]):
            z0, z1 = z_bins_2[zi], z_bins_2[zi+1]
            y0, y1 = y_bins_2[yi], y_bins_2[yi+1]
            x0, x1 = x_bins_2[xi], x_bins_2[xi+1]
            rid = zi * (num_bins_2[1] * num_bins_2[2]) + yi * num_bins_2[2] + xi

            sub_mask  = axon_mask[z0:z1, y0:y1, x0:x1]
            sub_skel  = axon_skel[z0:z1, y0:y1, x0:x1]
            vox_total = sub_mask.size
            vox_axon  = int(sub_mask.sum())
            vox_skel  = int(sub_skel.sum())  # skeleton voxels ~ length proxy (in voxel units)

            region_records.append({
                "region_id": rid,
                "axon_voxels": vox_axon,
                "region_voxels": vox_total,
                "axon_volume_fraction": vox_axon / vox_total if vox_total > 0 else 0.0,
                "skeleton_voxels": vox_skel
            })

df_axon_regions = pd.DataFrame(region_records)

# --------------------------
# 4B) COLOCALIZATION: ch1 blobs inside axon mask (CH2)
# --------------------------
def _in_bounds(z, y, x, shape):
    return (0 <= z < shape[0]) and (0 <= y < shape[1]) and (0 <= x < shape[2])

in_axon = []
for zc, yc, xc in (blobs[:, :3] if len(blobs) > 0 else []):
    zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
    if _in_bounds(zz, yy, xx, axon_mask.shape):
        in_axon.append(bool(axon_mask[zz, yy, xx]))
    else:
        in_axon.append(False)

if len(blobs) > 0:
    df["in_axon_mask_ch2"] = in_axon
    coloc_summary = df.groupby("region_id").agg(
        blobs_total=("id", "size"),
        blobs_in_axon=("in_axon_mask_ch2", "sum")
    ).reset_index()
    coloc_summary["frac_blobs_in_axon"] = (
        coloc_summary["blobs_in_axon"] / coloc_summary["blobs_total"].replace(0, np.nan)
    )
else:
    coloc_summary = pd.DataFrame(columns=["region_id", "blobs_total", "blobs_in_axon", "frac_blobs_in_axon"])

# --------------------------
# 4C) SAVE CHANNEL-2 SUMMARIES
# --------------------------
df_axon_regions.to_csv("axon_regions_ch2.csv", index=False)
coloc_summary.to_csv("blobs_in_axon_by_region.csv", index=False)
print("Saved: axon_regions_ch2.csv and blobs_in_axon_by_region.csv")


# --------------------------
# 5B) NAPARI VISUALIZATION (CH2)
# --------------------------
viewer_2 = napari.Viewer()
viewer_2.add_image(image_2, name='Channel 2 (raw)')
viewer_2.add_labels(axon_mask.astype(np.uint8), name='Axon mask (Frangi + hysteresis)')
viewer_2.add_image(axon_skel.astype(np.uint8), name='Axon skeleton', blending='additive', contrast_limits=(0,1))


# --------------------------
# 6) MOVIES / OVERLAYS
# --------------------------
# CH1 video (raw)
img_norm = to_uint8_stack(image)
frames = [img_norm[z] for z in range(img_norm.shape[0])]

try:
    imageio.mimsave('lysosome_stack.mp4', frames, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack.mp4')
except TypeError:
    imageio.mimsave('lysosome_stack.gif', frames, fps=8)
    print('Saved: lysosome_stack.gif')

# CH1 video with blobs overlay
frames_with_blobs = []
for z in range(img_norm.shape[0]):
    frame = img_norm[z].copy()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    # Draw blobs at this z-slice
    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            radius = int(round(b[3]))
            cv2.circle(frame_rgb, (x, y), radius, (0, 255, 255), 2)  # yellow
    frames_with_blobs.append(frame_rgb)

try:
    imageio.mimsave('lysosome_stack_blobs.mp4', frames_with_blobs, fps=8, format='FFMPEG')
    print('Saved: lysosome_stack_blobs.mp4')
except TypeError:
    imageio.mimsave('lysosome_stack_blobs.gif', frames_with_blobs, fps=8)
    print('Saved: lysosome_stack_blobs.gif')

# CH2 base stack (raw)
img_norm_2 = to_uint8_stack(image_2)
frames_2 = [img_norm_2[z2] for z2 in range(img_norm_2.shape[0])]
try:
    imageio.mimsave('axon_stack_ch2.mp4', frames_2, fps=8, format='FFMPEG')
    print("Saved: axon_stack_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_stack_ch2.gif', frames_2, fps=8)
    print("Saved: axon_stack_ch2.gif")

# CH2 overlay with axon mask (colored)
frames_mask_overlay = []
for z2 in range(img_norm_2.shape[0]):
    base = cv2.cvtColor(img_norm_2[z2], cv2.COLOR_GRAY2BGR)
    mask = (axon_mask[z2].astype(np.uint8) * 255)
    color_mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(base, 1.0, color_mask, 0.35, 0.0)
    frames_mask_overlay.append(overlay)

try:
    imageio.mimsave('axon_mask_overlay_ch2.mp4', frames_mask_overlay, fps=8, format='FFMPEG')
    print("Saved: axon_mask_overlay_ch2.mp4")
except TypeError:
    imageio.mimsave('axon_mask_overlay_ch2.gif', frames_mask_overlay, fps=8)
    print("Saved: axon_mask_overlay_ch2.gif")


# --------------------------
# 7) RUN VIEWERS
# --------------------------
napari.run()